<a href="https://colab.research.google.com/github/AgustinBustos/Tesis_Bella_Bonilla_Bustos/blob/main/try.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#falta stansarizar columnas
#wtf solo con elo en el modelo logistico parece que hace overfitting segun el modelo ridge
#creo que lo que sucede es que ell logistico hace overfitting en el de 2019 porque hay pocos datos, mientras que en el total no hace overfitting 
# ,y por lo tanto a lo mejor el ridge no hace falta en el full
#el problema que ya voy viendo es que el sesgo temporal del random split deberia ser alto, y por lo tanto deberia dar una acuracy bastante alta
#tengo que guardar las predicciones en otro data set
#hacer grafico del ridge en 2019 pero en ridge full
#deberia hacer mi libreria de nn
#sos un paja, deberias estandarizar
#primero hagamos un analisis piola con modelos tranca y despues le metemos nn que quiero hacer una mini libreria
#hacer modelo logistico temporal rapido
#no se que verga 
#no entiendo los problemas con la version 2step artificial
#que el factor cyclic sea un mejor que el atac def me da esperanza
#forzar el overfitting metiendo factores
#el factor tiene banda de volatilidad de entrenamiento a diferencia del otro
#piola la relacion entre la q de factores, el tam;o de la red y el overfitting
#tengo que duplicar el pands de entrenmiento para que la red me quede simetrica
# voy a tratar el accuracy del attack def
#me equivoque un poco en el de los factores, no puedo extender las filas para crear simetria, para
#que suceda eso tiene que haber 2 variables de output
#como solo hay un score de resultado, entonces no hace falta tener una sola matriz de factores 
#ahora es psible tener 2 matrices de fctores, factores de ataque y factores e defensa
#el chequeo de accuracy de ataque y defensa lo tengo que ponr en el predict2
#tengo que cambiar la verga de los y en matriz o en vector
#en atack y def voy a usar el 2019 como test
#puedo explorar como poner restricciones de regularizacion
#deberia hacer el dual fire
#tengo que hacer un trainer de red estocastico o algo para que sea mas consistente en el training
#tengo que hacer un modelo simple con attack, def, y elo
#hice una cobinacion muy simple con elo, del modelo ayd, pero puedo complicarla
#puedo hacer la version nn del elo creando un solo factor
#un factor deberia ser peor que elo?
#estoy cagado que haya data lekeage cuando defino la variable tiempo
#la funcion de la cantidad de filas es super importante, pero todavia no voy a trabajar con eso, simplemente uso bac=17
#deberia hacer un scatter entre factor1 y elo
#chequear errores con un factor da igual al elo
#me gustaria hacer la version hxh del elo
#ayd da lo mismo que un factor solo, pienso overfittear el tema para que de mas bajo
#creo que el bstract tiene volatilidad de entrenamiento
#hacer un scatter con y el test error y x eel train eror
#hay muchos bagging posibles fit and bag, o bag by val set
#hacer scatter de val error vs test error
#ver como va cambiando el error del modelo promedio cuando cambio el conjunto de modelos que uso segun el val error

# importo

In [1]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML
import plotly.express as px
import torch
import copy


#guarda con las warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
df1 = pd.read_csv('/content/drive/MyDrive/Ideas/operation bet/readyBenter_ (2).csv')

df1['y'] = 0
df1.loc[df1['team_score'] > df1['opp_score'],'y']=1
df1 = df1[['Home Team', 'Away Team','season', 'week','elo1_pre', 'elo2_pre', 'elo_prob1','Home Odds Open','y','team_score','opp_score']]
df1['Home Odds Open']=1/df1['Home Odds Open']
df1

,Home Team,Away Team,season,week,elo1_pre,elo2_pre,elo_prob1,Home Odds Open,y,team_score,opp_score
0,Pittsburgh Steelers,Miami Dolphins,NaN,1.0,NaN,NaN,NaN,0.555556,1,28.0,17.0
1,Carolina Panthers,Atlanta Falcons,2006.0,1.0,1595.288000,1492.786000,0.723966,0.696970,0,6.0,20.0
2,Tampa Bay Buccaneers,Baltimore Ravens,2006.0,1.0,1520.605000,1486.653000,0.638674,0.615385,0,0.0,27.0
3,New England Patriots,Buffalo Bills,2006.0,1.0,1603.944000,1474.302000,0.754072,0.851852,1,19.0,17.0
4,Green Bay Packers,Chicago Bears,2006.0,1.0,1464.766000,1515.842000,0.520028,0.357143,0,0.0,26.0
...,...,...,...,...,...,...,...,...,...,...,...
3796,Los Angeles Rams,New York Giants,2020.0,4.0,1575.113707,1348.285082,0.842891,0.862069,1,17.0,9.0
3797,San Francisco 49ers,Philadelphia Eagles,2020.0,4.0,1615.670408,1460.006767,0.780784,0.729927,0,20.0,25.0
3798,Miami Dolphins,Seattle Seahawks,2020.0,4.0,1435.437716,1605.437417,0.353332,0.281690,0,23.0,31.0
3799,Green Bay Packers,Atlanta Falcons,2020.0,4.0,1641.976946,1481.164708,0.785815,0.775194,1,30.0,16.0


In [3]:
elo=pd.read_csv('/content/drive/MyDrive/Ideas/operation bet/nfl_elo.csv')
elo

,date,season,neutral,playoff,team1,team2,elo1_pre,elo2_pre,elo_prob1,elo_prob2,elo1_post,elo2_post,qbelo1_pre,qbelo2_pre,qb1,qb2,qb1_value_pre,qb2_value_pre,qb1_adj,qb2_adj,qbelo_prob1,qbelo_prob2,qb1_game_value,qb2_game_value,qb1_value_post,qb2_value_post,qbelo1_post,qbelo2_post,score1,score2
0,1920-09-26,1920,0,NaN,RII,STP,1503.947,1300.000,0.824651,0.175349,1516.108,1287.838,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48.0,0.0
1,1920-10-03,1920,0,NaN,CBD,PTQ,1504.688,1300.000,0.825267,0.174733,1516.803,1287.885,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48.0,0.0
2,1920-10-03,1920,0,NaN,RII,MUN,1516.108,1478.004,0.644171,0.355829,1542.135,1451.977,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.0,0.0
3,1920-10-03,1920,0,NaN,DAY,COL,1493.002,1504.908,0.575819,0.424181,1515.434,1482.475,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,0.0
4,1920-10-03,1920,0,NaN,RCH,ABU,1503.420,1300.000,0.824212,0.175788,1510.934,1292.486,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16805,2021-01-17,2020,0,d,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16806,2021-01-17,2020,0,d,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16807,2021-01-24,2020,0,c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16808,2021-01-24,2020,0,c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
#primero simplifico el dataset, no quarter back, guarda, le saque el score
elo1=elo[['season','team1','team2','elo1_pre','elo2_pre','elo_prob1','score1','score2']].copy()
elo1['y']=0
elo1.loc[elo1['score1']>elo1['score2'],'y']=1
elo1=elo1.dropna()
elo1.drop(columns=['score1','score2'],inplace=True)

#trabajo con 2019
elo2=elo1[elo1['season']==2019].copy()
elo2.drop(columns='season',inplace=True)
elo1.to_csv('elo1.csv',index=False)
elo2.to_csv('elo2.csv',index=False)

elo1

,season,team1,team2,elo1_pre,elo2_pre,elo_prob1,y
0,1920,RII,STP,1503.947000,1300.000000,0.824651,1
1,1920,CBD,PTQ,1504.688000,1300.000000,0.825267,1
2,1920,RII,MUN,1516.108000,1478.004000,0.644171,1
3,1920,DAY,COL,1493.002000,1504.908000,0.575819,1
4,1920,RCH,ABU,1503.420000,1300.000000,0.824212,1
...,...,...,...,...,...,...,...
16599,2020,CHI,IND,1561.028166,1513.543373,0.656450,0
16600,2020,OAK,BUF,1472.589693,1551.165405,0.480473,0
16601,2020,SF,PHI,1615.670408,1460.006767,0.780784,0
16602,2020,KC,NE,1717.000270,1591.218959,0.749927,1


## train y test

In [5]:
#separo en train y test
spliter=np.random.randint(2, size=len(elo1))
elo1['split']=spliter
elo2=elo1[elo1['season']==2019].copy()
elo2.drop(columns='season',inplace=True)
elo2

,team1,team2,elo1_pre,elo2_pre,elo_prob1,y,split
16274,CHI,GB,1588.897931,1455.131370,0.758449,0,1
16275,NYJ,BUF,1384.966442,1448.423682,0.502220,0,0
16276,MIN,ATL,1538.423954,1520.315825,0.617372,1,1
16277,CAR,LAR,1519.378772,1598.015666,0.480385,0,0
16278,JAX,KC,1455.249073,1602.077427,0.384370,0,1
...,...,...,...,...,...,...,...
16536,KC,HOU,1694.572097,1552.346950,0.767256,1,1
16537,GB,SEA,1631.858399,1579.312894,0.662990,1,1
16538,KC,TEN,1707.523977,1662.579362,0.653145,1,0
16539,SF,GB,1651.872498,1643.322690,0.604293,1,0


## variables

In [6]:
x=['team1','team2','elo1_pre', 'elo2_pre', 'elo_prob1']
xelo=['elo1_pre', 'elo2_pre', 'elo_prob1']
xteam=[ 'team1_ARI', 'team1_ATL',
       'team1_BAL', 'team1_BUF', 'team1_CAR', 'team1_CHI', 'team1_CIN',
       'team1_CLE', 'team1_DAL', 'team1_DEN', 'team1_DET', 'team1_GB',
       'team1_HOU', 'team1_IND', 'team1_JAX', 'team1_KC', 'team1_LAC',
       'team1_LAR', 'team1_MIA', 'team1_MIN', 'team1_NE', 'team1_NO',
       'team1_NYG', 'team1_NYJ', 'team1_OAK', 'team1_PHI', 'team1_PIT',
       'team1_SEA', 'team1_SF', 'team1_TB', 'team1_TEN', 'team1_WSH',
       'team2_ARI', 'team2_ATL', 'team2_BAL', 'team2_BUF', 'team2_CAR',
       'team2_CHI', 'team2_CIN', 'team2_CLE', 'team2_DAL', 'team2_DEN',
       'team2_DET', 'team2_GB', 'team2_HOU', 'team2_IND', 'team2_JAX',
       'team2_KC', 'team2_LAC', 'team2_LAR', 'team2_MIA', 'team2_MIN',
       'team2_NE', 'team2_NO', 'team2_NYG', 'team2_NYJ', 'team2_OAK',
       'team2_PHI', 'team2_PIT', 'team2_SEA', 'team2_SF', 'team2_TB',
       'team2_TEN', 'team2_WSH']
teams=[ 'team1_ARI', 'team1_ATL',
       'team1_BAL', 'team1_BUF', 'team1_CAR', 'team1_CHI', 'team1_CIN',
       'team1_CLE', 'team1_DAL', 'team1_DEN', 'team1_DET', 'team1_GB',
       'team1_HOU', 'team1_IND', 'team1_JAX', 'team1_KC', 'team1_LAC',
       'team1_LAR', 'team1_MIA', 'team1_MIN', 'team1_NE', 'team1_NO',
       'team1_NYG', 'team1_NYJ', 'team1_OAK', 'team1_PHI', 'team1_PIT',
       'team1_SEA', 'team1_SF', 'team1_TB', 'team1_TEN', 'team1_WSH']
teams=[s.replace("team1_","") for s in teams]
xteam2=['Home Team_Arizona Cardinals', 'Home Team_Atlanta Falcons',
       'Home Team_Baltimore Ravens', 'Home Team_Buffalo Bills',
       'Home Team_Carolina Panthers', 'Home Team_Chicago Bears',
       'Home Team_Cincinnati Bengals', 'Home Team_Cleveland Browns',
       'Home Team_Dallas Cowboys', 'Home Team_Denver Broncos',
       'Home Team_Detroit Lions', 'Home Team_Green Bay Packers',
       'Home Team_Houston Texans', 'Home Team_Indianapolis Colts',
       'Home Team_Jacksonville Jaguars', 'Home Team_Kansas City Chiefs',
       'Home Team_Los Angeles Chargers', 'Home Team_Los Angeles Rams',
       'Home Team_Miami Dolphins', 'Home Team_Minnesota Vikings',
       'Home Team_New England Patriots', 'Home Team_New Orleans Saints',
       'Home Team_New York Giants', 'Home Team_New York Jets',
       'Home Team_Oakland Raiders', 'Home Team_Philadelphia Eagles',
       'Home Team_Pittsburgh Steelers', 'Home Team_San Francisco 49ers',
       'Home Team_Seattle Seahawks', 'Home Team_Tampa Bay Buccaneers',
       'Home Team_Tennessee Titans', 'Home Team_Washington Football Team',
       'Away Team_Arizona Cardinals', 'Away Team_Atlanta Falcons',
       'Away Team_Baltimore Ravens', 'Away Team_Buffalo Bills',
       'Away Team_Carolina Panthers', 'Away Team_Chicago Bears',
       'Away Team_Cincinnati Bengals', 'Away Team_Cleveland Browns',
       'Away Team_Dallas Cowboys', 'Away Team_Denver Broncos',
       'Away Team_Detroit Lions', 'Away Team_Green Bay Packers',
       'Away Team_Houston Texans', 'Away Team_Indianapolis Colts',
       'Away Team_Jacksonville Jaguars', 'Away Team_Kansas City Chiefs',
       'Away Team_Los Angeles Chargers', 'Away Team_Los Angeles Rams',
       'Away Team_Miami Dolphins', 'Away Team_Minnesota Vikings',
       'Away Team_New England Patriots', 'Away Team_New Orleans Saints',
       'Away Team_New York Giants', 'Away Team_New York Jets',
       'Away Team_Oakland Raiders', 'Away Team_Philadelphia Eagles',
       'Away Team_Pittsburgh Steelers', 'Away Team_San Francisco 49ers',
       'Away Team_Seattle Seahawks', 'Away Team_Tampa Bay Buccaneers',
       'Away Team_Tennessee Titans', 'Away Team_Washington Football Team']
      
len(xteam2)

64

# funciones

In [7]:
def rateOfSucc(df,predName):
  dfCopy=df.copy()
  dfCopy['success']=0
  dfCopy.loc[dfCopy['y']==dfCopy[predName],'success']=1
  return len(dfCopy.loc[dfCopy['success']==1])/len(dfCopy)







In [31]:
def mse(t1,t2):
  diff = t1 - t2
  return torch.sum(diff * diff) / diff.numel()
  

#API

## fit

In [12]:
#trabajo con variables globales
def fit(parametros,epochNum=5000,lr=0.1,graph=True,wd=0):
  parameters=parametros()
  
  optimizer = torch.optim.Adam(parameters, lr=lr,weight_decay=wd)
  trainError=[]
  valError=[]

  bestError=mse(model(xv,parameters), yv).detach().numpy()
  bestParams=[param.clone().detach() for param in parameters]

  for epoch in range(epochNum):
    #entreno
    yhat = model(xt,parameters)
    loss = mse(yhat, yt)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    #guardovalores
    val=mse(model(xv,parameters), yv).detach().numpy()
    trainError.append(loss.detach().numpy())
    valError.append(val)

    #guardo el mejor model
    if val<bestError:
      bestError=val
      bestParams=[param.clone().detach() for param in parameters]

    
  if graph:
    px.line(x=[t for t in range(epochNum)]+[t for t in range(epochNum)],
                          y=trainError+valError).show()



  return {'bestError':bestError,
          'bestParams':bestParams,
          'graph':px.line(x=[t for t in range(epochNum)]+[t for t in range(epochNum)],
                          y=trainError+valError)}
  


In [13]:
#trabajo con variables globales
def fitDF(parametros,epochNum=5000,lr=0.1,graph=True,wd=0):
  parameters=parametros()
  
  optimizer1 = torch.optim.Adam([parameters[0]], lr=lr)
  optimizer2 = torch.optim.Adam(parameters[1:], lr=lr,weight_decay=wd)
  trainError=[]
  valError=[]

  bestError=mse(model(xv,parameters), yv).detach().numpy()
  bestParams=[param.clone().detach() for param in parameters]

  for epoch in range(epochNum):
    #entreno
    yhat = model(xt,parameters)
    loss = mse(yhat, yt)
    optimizer1.zero_grad()
    loss.backward()
    optimizer1.step()

    #red
    yhat = model(xt,parameters)
    loss = mse(yhat, yt)
    optimizer2.zero_grad()
    loss.backward()
    optimizer2.step()

    #guardovalores
    val=mse(model(xv,parameters), yv).detach().numpy()
    trainError.append(loss.detach().numpy())
    valError.append(val)

    #guardo el mejor model
    if val<bestError:
      bestError=val
      bestParams=[param.clone().detach() for param in parameters]

    
  if graph:
    px.line(x=[t for t in range(epochNum)]+[t for t in range(epochNum)],
                          y=trainError+valError).show()



  return {'bestError':bestError,
          'bestParams':bestParams,
          'graph':px.line(x=[t for t in range(epochNum)]+[t for t in range(epochNum)],
                          y=trainError+valError)}
  


## stoch

In [14]:
#idea es bajar la volatilidad de entrenamiento
def stochFit(fit,stochNum=20,epochNum=5000,lr=0.1,graph=True,wd=0):
  
  bestModel=fit(parametros,epochNum=epochNum,lr=lr,graph=False,wd=wd)
  for i in range(stochNum):
    
    currentModel=fit(parametros,epochNum=epochNum,lr=lr,graph=False,wd=wd)
    if currentModel['bestError']<bestModel['bestError']:
      bestModel=currentModel
  if graph:
    bestModel['graph'].show()


  return bestModel

In [15]:
def fitindebag(fit,stochNum=20,epochNum=5000,lr=0.1,graph=True,wd=0):
  models=[]
  
  
  for i in range(stochNum):
    
    models.append(fit(parametros,epochNum=epochNum,lr=lr,graph=False,wd=wd))
    


  return models

# --------------------Data Analisis----------------

# elo score

In [ ]:
anal=elo.loc[elo['season']==2018]
anal['scoreDiff']=anal['score1']-anal['score2']
px.scatter(anal,x='elo_prob1',y='scoreDiff',trendline='ols').show()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



# market bias by importance

In [ ]:
aass=np.linspace(0.2,1)
succ=[]

elo2['yHat_elo2']=0
elo2.loc[elo2['elo_prob1']>0.5,'yHat_elo2']=1
for a in aass:

  
  succ.append(rateOfSucc(elo2.loc[elo2['elo_prob1']<a],'yHat_elo2'))

px.line(x=aass,y=succ).show()

In [ ]:
aass=np.linspace(0.2,1)
succ=[]
succ2=[]

df1['yHat_elo2']=0
df1.loc[df1['elo_prob1']>0.5,'yHat_elo2']=1
df1['yHat_odds']=0
df1.loc[df1['Home Odds Open']>0.5,'yHat_odds']=1
for a in aass:

  
  succ.append(rateOfSucc(df1.loc[df1['elo_prob1']<a],'yHat_elo2'))
  succ2.append(rateOfSucc(df1.loc[df1['Home Odds Open']<a],'yHat_odds'))

px.line(x=list(aass)+list(aass),y=succ+succ2).show()

In [ ]:
df1

,Home Team,Away Team,season,elo1_pre,elo2_pre,elo_prob1,Home Odds Open,y,yHat_elo2
0,Pittsburgh Steelers,Miami Dolphins,NaN,NaN,NaN,NaN,0.555556,1,0
1,Carolina Panthers,Atlanta Falcons,2006.0,1595.288000,1492.786000,0.723966,0.696970,0,1
2,Tampa Bay Buccaneers,Baltimore Ravens,2006.0,1520.605000,1486.653000,0.638674,0.615385,0,1
3,New England Patriots,Buffalo Bills,2006.0,1603.944000,1474.302000,0.754072,0.851852,1,1
4,Green Bay Packers,Chicago Bears,2006.0,1464.766000,1515.842000,0.520028,0.357143,0,1
...,...,...,...,...,...,...,...,...,...
3796,Los Angeles Rams,New York Giants,2020.0,1575.113707,1348.285082,0.842891,0.862069,1,1
3797,San Francisco 49ers,Philadelphia Eagles,2020.0,1615.670408,1460.006767,0.780784,0.729927,0,1
3798,Miami Dolphins,Seattle Seahawks,2020.0,1435.437716,1605.437417,0.353332,0.281690,0,0
3799,Green Bay Packers,Atlanta Falcons,2020.0,1641.976946,1481.164708,0.785815,0.775194,1,1


# ----------------To Beat------------------

# ELO

## predict

In [ ]:
predict1=elo1[['y','split','season']].copy()


predict2=elo2[['y','split']].copy()


#hago copias
elo2Aux=elo2.copy()
elo1Aux=elo1.copy()
predict1


,y,split,season
0,1,0,1920
1,1,0,1920
2,1,0,1920
3,1,1,1920
4,1,0,1920
...,...,...,...
16599,0,0,2020
16600,0,1,2020
16601,0,1,2020
16602,1,1,2020


## 2019

In [ ]:
#trabajo con 2019, primera prediccion usando probs
#hago una copia

elo2Aux['y_hat_eloProb']=0
elo2Aux.loc[elo2Aux['elo_prob1']>0.5,'y_hat_eloProb']=1
#elo rocho
elo2Aux['y_hat_eloProb2']=0
elo2Aux.loc[elo2Aux['elo1_pre']>elo2Aux['elo2_pre'],'y_hat_eloProb2']=1
predict2['y_hat_eloProb']=elo2Aux['y_hat_eloProb']
predict2['y_hat_eloProb2']=elo2Aux['y_hat_eloProb2']
display(predict2)
print(rateOfSucc(predict2,'y_hat_eloProb2'))
print(rateOfSucc(predict2,'y_hat_eloProb'))

,y,split,y_hat_eloProb,y_hat_eloProb2
16274,0,0,1,1
16275,0,0,1,0
16276,1,0,1,1
16277,0,1,0,0
16278,0,1,0,0
...,...,...,...,...
16536,1,0,1,1
16537,1,1,1,1
16538,1,0,1,1
16539,1,0,1,1


0.6404494382022472
0.6404494382022472


## full

In [ ]:
#full dataset
elo3=elo1.dropna()

#trabajo con 2019, primera prediccion usando probs
elo3['y_hat_eloProb']=0
elo3.loc[elo3['elo_prob1']>0.5,'y_hat_eloProb']=1
#elo rocho
elo3['y_hat_eloProb2']=0
elo3.loc[elo3['elo1_pre']>elo3['elo2_pre'],'y_hat_eloProb2']=1

#meto en el predict
predict1['y_hat_eloProb2']=elo3['y_hat_eloProb2']
print(rateOfSucc(predict1,'y_hat_eloProb2'))

predict1['y_hat_eloProb']=elo3['y_hat_eloProb']
print(rateOfSucc(predict1,'y_hat_eloProb'))
predict1

0.6459889183329318
0.6599614550710672


,y,split,season,y_hat_eloProb2,y_hat_eloProb
0,1,0,1920,1,1
1,1,0,1920,1,1
2,1,0,1920,1,1
3,1,1,1920,0,1
4,1,0,1920,1,1
...,...,...,...,...,...
16599,0,0,2020,1,1
16600,0,1,2020,0,0
16601,0,1,2020,1,1
16602,1,1,2020,1,1


# # ----------------Attempts-------------

# logistic reg

Le esta pegando muy al palo, le gana al elo en el full dataset, pero pierde en 2019

In [ ]:
from sklearn.linear_model import LogisticRegression

## 2019

In [ ]:
eloTec=pd.get_dummies(elo2, columns=['team1',	'team2'])
eloTecTrain=eloTec.loc[eloTec['split']==0]
eloTecTest=eloTec.loc[eloTec['split']==1]
eloTecTest

,elo1_pre,elo2_pre,elo_prob1,y,split,team1_ARI,team1_ATL,team1_BAL,team1_BUF,team1_CAR,team1_CHI,team1_CIN,team1_CLE,team1_DAL,team1_DEN,team1_DET,team1_GB,team1_HOU,team1_IND,team1_JAX,team1_KC,team1_LAC,team1_LAR,team1_MIA,team1_MIN,team1_NE,team1_NO,team1_NYG,team1_NYJ,team1_OAK,team1_PHI,team1_PIT,team1_SEA,team1_SF,team1_TB,team1_TEN,team1_WSH,team2_ARI,team2_ATL,team2_BAL,team2_BUF,team2_CAR,team2_CHI,team2_CIN,team2_CLE,team2_DAL,team2_DEN,team2_DET,team2_GB,team2_HOU,team2_IND,team2_JAX,team2_KC,team2_LAC,team2_LAR,team2_MIA,team2_MIN,team2_NE,team2_NO,team2_NYG,team2_NYJ,team2_OAK,team2_PHI,team2_PIT,team2_SEA,team2_SF,team2_TB,team2_TEN,team2_WSH
16274,1588.897931,1455.131370,0.758449,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
16279,1581.513559,1441.021792,0.765470,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
16284,1444.772669,1422.631825,0.622841,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
16286,1546.637605,1442.725536,0.725585,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
16287,1640.856488,1572.193220,0.683397,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16532,1692.254223,1596.844105,0.715733,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
16534,1629.192628,1626.889910,0.595663,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
16535,1752.748592,1614.977082,0.762647,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
16537,1631.858399,1579.312894,0.662990,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [ ]:
y=eloTecTrain.loc[:,'y'].to_numpy()
X=eloTecTrain.loc[:,xelo+xteam].to_numpy()
clf = LogisticRegression().fit(X, y)

predict2['logistic']=clf.predict(eloTec.loc[:,xelo+xteam].to_numpy())
predict2


,y,split,y_hat_eloProb,y_hat_eloProb2,logistic
16274,0,1,1,1,1
16275,0,0,1,0,0
16276,1,0,1,1,1
16277,0,0,0,0,0
16278,0,0,0,0,0
...,...,...,...,...,...
16536,1,0,1,1,1
16537,1,1,1,1,1
16538,1,1,1,1,1
16539,1,0,1,1,1


In [ ]:
 # el train da 0.76, si separo en 1/2 es 0.82 y 0.63
 print(rateOfSucc(predict2.loc[predict2['split']==0],'logistic'))
 print(rateOfSucc(predict2.loc[predict2['split']==1],'logistic'))

0.8283582089552238
0.5939849624060151


## full

In [ ]:
eloTec=pd.get_dummies(elo1, columns=['team1',	'team2','season'])
eloTecTrain=eloTec.loc[eloTec['split']==0]
eloTecTest=eloTec.loc[eloTec['split']==1]
eloTec

,elo1_pre,elo2_pre,elo_prob1,y,split,team1_AKR,team1_ARI,team1_ATL,team1_BAL,team1_BBA,team1_BCL,team1_BDA,team1_BFF,team1_BKN,team1_BRL,team1_BUF,team1_BYK,team1_CAR,team1_CBD,team1_CHB,team1_CHI,team1_CHL,team1_CHT,team1_CIB,team1_CIN,team1_CLE,team1_CLI,team1_COL,team1_CRA,team1_CRP,team1_CST,team1_CTI,team1_DAL,team1_DAY,team1_DEN,team1_DET,team1_DHR,team1_DPN,team1_DTI,team1_DTX,...,season_1981,season_1982,season_1983,season_1984,season_1985,season_1986,season_1987,season_1988,season_1989,season_1990,season_1991,season_1992,season_1993,season_1994,season_1995,season_1996,season_1997,season_1998,season_1999,season_2000,season_2001,season_2002,season_2003,season_2004,season_2005,season_2006,season_2007,season_2008,season_2009,season_2010,season_2011,season_2012,season_2013,season_2014,season_2015,season_2016,season_2017,season_2018,season_2019,season_2020
0,1503.947000,1300.000000,0.824651,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1504.688000,1300.000000,0.825267,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1516.108000,1478.004000,0.644171,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1493.002000,1504.908000,0.575819,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1503.420000,1300.000000,0.824212,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16599,1561.028166,1513.543373,0.656450,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
16600,1472.589693,1551.165405,0.480473,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
16601,1615.670408,1460.006767,0.780784,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
16602,1717.000270,1591.218959,0.749927,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [ ]:
y=eloTecTrain.loc[:,'y'].to_numpy()
X=eloTecTrain.drop(columns=['y',	'split']).to_numpy()
clf = LogisticRegression().fit(X, y)

predict1['logistic']=clf.predict(eloTec.drop(columns=['y',	'split']).to_numpy())
predict1

,y,split,y_hat_eloProb2,y_hat_eloProb,logistic
0,1,1,1,1,1
1,1,0,1,1,1
2,1,0,1,1,1
3,1,1,0,1,1
4,1,1,1,1,1
...,...,...,...,...,...
16599,0,1,1,1,1
16600,0,1,0,0,0
16601,0,0,1,1,1
16602,1,0,1,1,1


In [ ]:
 # con el full data set da 
 print(rateOfSucc(predict1.loc[predict1['split']==0],'logistic'))
 print(rateOfSucc(predict1.loc[predict1['split']==1],'logistic'))

0.6536791314837154
0.6692326196776521


# fastai MAL

In [ ]:
! [ -e /content ] && pip install -Uqq fastai 

     |████████████████████████████████| 189 kB 13.5 MB/s 
     |████████████████████████████████| 56 kB 3.9 MB/s 


In [ ]:
from fastai.tabular.all import *

In [ ]:
elo2

,team1,team2,elo1_pre,elo2_pre,elo_prob1,y,split
16274,CHI,GB,1588.897931,1455.131370,0.758449,0,1
16275,NYJ,BUF,1384.966442,1448.423682,0.502220,0,1
16276,MIN,ATL,1538.423954,1520.315825,0.617372,1,1
16277,CAR,LAR,1519.378772,1598.015666,0.480385,0,1
16278,JAX,KC,1455.249073,1602.077427,0.384370,0,0
...,...,...,...,...,...,...,...
16536,KC,HOU,1694.572097,1552.346950,0.767256,1,1
16537,GB,SEA,1631.858399,1579.312894,0.662990,1,0
16538,KC,TEN,1707.523977,1662.579362,0.653145,1,1
16539,SF,GB,1651.872498,1643.322690,0.604293,1,0


In [ ]:
cat_names = ['team1', 'team2']
cont_names = ['elo1_pre',	'elo2_pre',	'elo_prob1']
procs = [FillMissing, Categorify, Normalize]
dep_var = 'y'

dls = TabularDataLoaders.from_df(df=elo2.loc[:,x+['y']], y_names=dep_var,
    cat_names = cat_names,
    cont_names = cont_names,
    procs =procs)

In [ ]:
splits = RandomSplitter(valid_pct=0.5)(range_of(elo2.loc[:,x+['y']]))

In [ ]:
to = TabularPandas(elo2.loc[:,x+['y']], procs=procs,
                   cat_names = cat_names,
                   cont_names = cont_names,
                   y_names=dep_var,
                   splits=splits)

In [ ]:
dls = to.dataloaders(bs=2)
dls.show_batch()

,team1,team2,elo1_pre,elo2_pre,elo_prob1,y
0,DET,GB,1360.450445,1625.761358,0.239927,0.0
1,DET,MIN,1506.926758,1588.285769,0.476475,0.0


In [ ]:
learn = tabular_learner(dls, metrics=accuracy)

In [ ]:
learn.fit_one_cycle(10)

epoch,train_loss,valid_loss,accuracy,time
0,0.470469,0.567108,0.458647,00:01
1,0.374385,0.265822,0.458647,00:01
2,0.307982,0.277588,0.458647,00:00
3,0.264494,0.281301,0.458647,00:00
4,0.270178,0.369390,0.458647,00:00
5,0.257176,0.310097,0.458647,00:00
6,0.246639,0.282103,0.458647,00:01
7,0.247853,0.272751,0.458647,00:01
8,0.222284,0.271077,0.458647,00:01
9,0.236088,0.294351,0.458647,00:01


In [ ]:
learn.show_results()

,team1,team2,elo1_pre,elo2_pre,elo_prob1,y,y_pred
0,4.0,7.0,-0.329710,-1.312250,0.784760,1.0,0.388267
1,12.0,6.0,0.542233,0.446865,0.146537,1.0,0.434838


# ridge

Le esta pegando muy al palo, le gana al elo en el full dataset, pero pierde en 2019

In [ ]:
from sklearn.linear_model import RidgeClassifier

## 2019

In [ ]:
eloTec=pd.get_dummies(elo2, columns=['team1',	'team2'])
eloTecTrain=eloTec.loc[eloTec['split']==0]
eloTecTest=eloTec.loc[eloTec['split']==1]
eloTecTest

,elo1_pre,elo2_pre,elo_prob1,y,split,team1_ARI,team1_ATL,team1_BAL,team1_BUF,team1_CAR,team1_CHI,team1_CIN,team1_CLE,team1_DAL,team1_DEN,team1_DET,team1_GB,team1_HOU,team1_IND,team1_JAX,team1_KC,team1_LAC,team1_LAR,team1_MIA,team1_MIN,team1_NE,team1_NO,team1_NYG,team1_NYJ,team1_OAK,team1_PHI,team1_PIT,team1_SEA,team1_SF,team1_TB,team1_TEN,team1_WSH,team2_ARI,team2_ATL,team2_BAL,team2_BUF,team2_CAR,team2_CHI,team2_CIN,team2_CLE,team2_DAL,team2_DEN,team2_DET,team2_GB,team2_HOU,team2_IND,team2_JAX,team2_KC,team2_LAC,team2_LAR,team2_MIA,team2_MIN,team2_NE,team2_NO,team2_NYG,team2_NYJ,team2_OAK,team2_PHI,team2_PIT,team2_SEA,team2_SF,team2_TB,team2_TEN,team2_WSH
16274,1588.897931,1455.131370,0.758449,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
16279,1581.513559,1441.021792,0.765470,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
16284,1444.772669,1422.631825,0.622841,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
16286,1546.637605,1442.725536,0.725585,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
16287,1640.856488,1572.193220,0.683397,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16532,1692.254223,1596.844105,0.715733,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
16534,1629.192628,1626.889910,0.595663,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
16535,1752.748592,1614.977082,0.762647,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
16537,1631.858399,1579.312894,0.662990,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [ ]:
y=eloTecTrain.loc[:,'y'].to_numpy()
X=eloTecTrain.loc[:,xelo+xteam].to_numpy()

clf = RidgeClassifier(alpha=10).fit(X, y)

predict2['ridge']=clf.predict(eloTec.loc[:,xelo+xteam].to_numpy())
display(elo2)
# el train da 0.76, si separo en 1/2 es 0.82 y 0.63
print(rateOfSucc(predict2.loc[predict2['split']==0],'ridge'))
print(rateOfSucc(predict2.loc[predict2['split']==1],'ridge'))

,team1,team2,elo1_pre,elo2_pre,elo_prob1,y,split
16274,CHI,GB,1588.897931,1455.131370,0.758449,0,1
16275,NYJ,BUF,1384.966442,1448.423682,0.502220,0,0
16276,MIN,ATL,1538.423954,1520.315825,0.617372,1,0
16277,CAR,LAR,1519.378772,1598.015666,0.480385,0,0
16278,JAX,KC,1455.249073,1602.077427,0.384370,0,0
...,...,...,...,...,...,...,...
16536,KC,HOU,1694.572097,1552.346950,0.767256,1,0
16537,GB,SEA,1631.858399,1579.312894,0.662990,1,1
16538,KC,TEN,1707.523977,1662.579362,0.653145,1,1
16539,SF,GB,1651.872498,1643.322690,0.604293,1,0


0.8059701492537313
0.6390977443609023


In [ ]:
# voy a hacer un grafico para chequear el overfitting del modelo logistico
trainAccArray=[]
testAccArray=[]
net=np.logspace(-4, 4)
for regu in net:
  clf = RidgeClassifier(alpha=regu).fit(X, y)

  predict2['ridge']=clf.predict(eloTec.loc[:,xelo+xteam].to_numpy())
  
  # el train da 0.76, si separo en 1/2 es 0.82 y 0.63
  trainAccArray.append(rateOfSucc(predict2.loc[predict2['split']==0],'ridge'))
  testAccArray.append(rateOfSucc(predict2.loc[predict2['split']==1],'ridge'))
px.line(x=list(net)+list(net),
        y=testAccArray+trainAccArray,
        log_x=True).show()

## full

In [ ]:
eloTec=pd.get_dummies(elo1, columns=['team1',	'team2','season'])
eloTecTrain=eloTec.loc[eloTec['split']==0]
eloTecTest=eloTec.loc[eloTec['split']==1]
eloTec

,elo1_pre,elo2_pre,elo_prob1,y,split,team1_AKR,team1_ARI,team1_ATL,team1_BAL,team1_BBA,team1_BCL,team1_BDA,team1_BFF,team1_BKN,team1_BRL,team1_BUF,team1_BYK,team1_CAR,team1_CBD,team1_CHB,team1_CHI,team1_CHL,team1_CHT,team1_CIB,team1_CIN,team1_CLE,team1_CLI,team1_COL,team1_CRA,team1_CRP,team1_CST,team1_CTI,team1_DAL,team1_DAY,team1_DEN,team1_DET,team1_DHR,team1_DPN,team1_DTI,team1_DTX,...,season_1981,season_1982,season_1983,season_1984,season_1985,season_1986,season_1987,season_1988,season_1989,season_1990,season_1991,season_1992,season_1993,season_1994,season_1995,season_1996,season_1997,season_1998,season_1999,season_2000,season_2001,season_2002,season_2003,season_2004,season_2005,season_2006,season_2007,season_2008,season_2009,season_2010,season_2011,season_2012,season_2013,season_2014,season_2015,season_2016,season_2017,season_2018,season_2019,season_2020
0,1503.947000,1300.000000,0.824651,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1504.688000,1300.000000,0.825267,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1516.108000,1478.004000,0.644171,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1493.002000,1504.908000,0.575819,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1503.420000,1300.000000,0.824212,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16599,1561.028166,1513.543373,0.656450,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
16600,1472.589693,1551.165405,0.480473,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
16601,1615.670408,1460.006767,0.780784,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
16602,1717.000270,1591.218959,0.749927,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [ ]:
trainAccArray=[]
testAccArray=[]
net=np.logspace(-4, 10)
y=eloTecTrain.loc[:,'y'].to_numpy()
X=eloTecTrain.drop(columns=['y',	'split']).to_numpy()
for regu in net:
  clf = RidgeClassifier(alpha=regu).fit(X, y)

  predict1['ridge']=clf.predict(eloTec.drop(columns=['y',	'split']).to_numpy())
  
  # el train da 0.76, si separo en 1/2 es 0.82 y 0.63
  trainAccArray.append(rateOfSucc(predict1.loc[predict1['split']==0],'ridge'))
  testAccArray.append(rateOfSucc(predict1.loc[predict1['split']==1],'ridge'))
px.line(x=list(net)+list(net),
        y=testAccArray+trainAccArray,
        log_x=True).show()

# sk modular


Le esta pegando muy al palo, le gana al elo en el full dataset, pero pierde en 2019

In [ ]:
from sklearn.linear_model import RidgeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

## 2019

In [ ]:
eloTec=pd.get_dummies(elo2, columns=['team1',	'team2'])
eloTecTrain=eloTec.loc[eloTec['split']==0]
eloTecTest=eloTec.loc[eloTec['split']==1]

y=eloTecTrain.loc[:,'y'].to_numpy()
X=eloTecTrain.loc[:,xelo+xteam].to_numpy()

clf = RandomForestClassifier(n_estimators=100,min_samples_leaf=11).fit(X, y)

predict2['ridge']=clf.predict(eloTec.loc[:,xelo+xteam].to_numpy())

# el train da 0.76, si separo en 1/2 es 0.82 y 0.63
print(rateOfSucc(predict2.loc[predict2['split']==0],'ridge'))
print(rateOfSucc(predict2.loc[predict2['split']==1],'ridge'))

0.7014925373134329
0.5939849624060151


In [ ]:
# voy a hacer un grafico para chequear el overfitting del modelo logistico
trainAccArray=[]
testAccArray=[]
net=range(1,20)#np.logspace(-4, 4)
for regu in net:
  clf =RandomForestClassifier(n_estimators=1000,max_depth=regu).fit(X, y)

  predict2['ridge']=clf.predict(eloTec.loc[:,xelo+xteam].to_numpy())
  
  # el train da 0.76, si separo en 1/2 es 0.82 y 0.63
  trainAccArray.append(rateOfSucc(predict2.loc[predict2['split']==0],'ridge'))
  testAccArray.append(rateOfSucc(predict2.loc[predict2['split']==1],'ridge'))
px.line(x=list(net)+list(net),
        y=testAccArray+trainAccArray,
        log_x=True).show()


In [ ]:
#logistic 0.53
#ridge 0.61
# svc 0.62,0.63
#forest 0.63

## full

In [ ]:
from sklearn.linear_model import RidgeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
eloTec=pd.get_dummies(elo1, columns=['team1',	'team2','season'])
eloTecTrain=eloTec.loc[eloTec['split']==0]
eloTecTest=eloTec.loc[eloTec['split']==1]
eloTec

,elo1_pre,elo2_pre,elo_prob1,y,split,team1_AKR,team1_ARI,team1_ATL,team1_BAL,team1_BBA,team1_BCL,team1_BDA,team1_BFF,team1_BKN,team1_BRL,team1_BUF,team1_BYK,team1_CAR,team1_CBD,team1_CHB,team1_CHI,team1_CHL,team1_CHT,team1_CIB,team1_CIN,team1_CLE,team1_CLI,team1_COL,team1_CRA,team1_CRP,team1_CST,team1_CTI,team1_DAL,team1_DAY,team1_DEN,team1_DET,team1_DHR,team1_DPN,team1_DTI,team1_DTX,...,season_1981,season_1982,season_1983,season_1984,season_1985,season_1986,season_1987,season_1988,season_1989,season_1990,season_1991,season_1992,season_1993,season_1994,season_1995,season_1996,season_1997,season_1998,season_1999,season_2000,season_2001,season_2002,season_2003,season_2004,season_2005,season_2006,season_2007,season_2008,season_2009,season_2010,season_2011,season_2012,season_2013,season_2014,season_2015,season_2016,season_2017,season_2018,season_2019,season_2020
0,1503.947000,1300.000000,0.824651,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1504.688000,1300.000000,0.825267,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1516.108000,1478.004000,0.644171,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1493.002000,1504.908000,0.575819,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1503.420000,1300.000000,0.824212,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16599,1561.028166,1513.543373,0.656450,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
16600,1472.589693,1551.165405,0.480473,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
16601,1615.670408,1460.006767,0.780784,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
16602,1717.000270,1591.218959,0.749927,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [ ]:
trainAccArray=[]
testAccArray=[]
net=range(1,20)#np.logspace(-4, 10,num=5)
y=eloTecTrain.loc[:,'y'].to_numpy()
X=eloTecTrain.drop(columns=['y',	'split']).to_numpy()
for regu in net:
  clf = RandomForestClassifier(max_depth=regu,n_jobs=-1).fit(X, y)
  print(regu)

  predict1['ridge']=clf.predict(eloTec.drop(columns=['y',	'split']).to_numpy())
  
  # el train da 0.76, si separo en 1/2 es 0.82 y 0.63
  trainAccArray.append(rateOfSucc(predict1.loc[predict1['split']==0],'ridge'))
  testAccArray.append(rateOfSucc(predict1.loc[predict1['split']==1],'ridge'))
px.line(x=list(net)+list(net),
        y=testAccArray+trainAccArray,
        log_x=True).show()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [ ]:
#ridge 0.67
#svc 0.65, 0.66
#tree 0.66


# ---------------------custom for 2019-----------------

# no time to die

In [ ]:
#meto un sesgo temporal, leave one out donde entreno con casi todos los partidos de 2019 y testeo con un solo partido
# la idea es generalizar el elo, asi que no deberia entrar el elo en principio
#uso pytorch

## primero hago el cambio de pandas

In [8]:
trainingYear=2018
ayd=elo[['season','team1','team2','score1','score2']]
#dropeo los nans
ayd=ayd.dropna()
#hago una simetria
aydCopy=ayd.copy()
aydCopy['team1']=ayd['team2']
aydCopy['team2']=ayd['team1']
aydCopy['score1']=ayd['score2']
aydCopy['score2']=ayd['score1']
display(ayd.loc[ayd['season']==trainingYear])
aydCopy.loc[ayd['season']==trainingYear]

,season,team1,team2,score1,score2
16007,2018,PHI,ATL,18.0,12.0
16008,2018,IND,CIN,23.0,34.0
16009,2018,MIN,SF,24.0,16.0
16010,2018,MIA,TEN,27.0,20.0
16011,2018,NO,TB,40.0,48.0
...,...,...,...,...,...
16269,2018,NE,LAC,41.0,28.0
16270,2018,NO,PHI,20.0,14.0
16271,2018,NO,LAR,23.0,26.0
16272,2018,KC,NE,31.0,37.0


,season,team1,team2,score1,score2
16007,2018,ATL,PHI,12.0,18.0
16008,2018,CIN,IND,34.0,23.0
16009,2018,SF,MIN,16.0,24.0
16010,2018,TEN,MIA,20.0,27.0
16011,2018,TB,NO,48.0,40.0
...,...,...,...,...,...
16269,2018,LAC,NE,28.0,41.0
16270,2018,PHI,NO,14.0,20.0
16271,2018,LAR,NO,26.0,23.0
16272,2018,NE,KC,37.0,31.0


In [9]:
#concateno por columna
ayd1=pd.concat([ayd,aydCopy],axis=0)
#dropeo score sobrante
ayd1.drop(columns='score2',inplace=True)
ayd1.reset_index(inplace=True)
ayd1.drop(columns='index',inplace=True)
ayd1.loc[ayd1['season']==trainingYear]

,season,team1,team2,score1
16007,2018,PHI,ATL,18.0
16008,2018,IND,CIN,23.0
16009,2018,MIN,SF,24.0
16010,2018,MIA,TEN,27.0
16011,2018,NO,TB,40.0
...,...,...,...,...
32873,2018,LAC,NE,28.0
32874,2018,PHI,NO,14.0
32875,2018,LAR,NO,26.0
32876,2018,NE,KC,37.0


In [10]:
#creo que lo resolvi
#centro en 2019
ayd2=pd.get_dummies(ayd1[ayd1['season']==trainingYear], columns=['team1',	'team2'])
ayd2

,season,score1,team1_ARI,team1_ATL,team1_BAL,team1_BUF,team1_CAR,team1_CHI,team1_CIN,team1_CLE,team1_DAL,team1_DEN,team1_DET,team1_GB,team1_HOU,team1_IND,team1_JAX,team1_KC,team1_LAC,team1_LAR,team1_MIA,team1_MIN,team1_NE,team1_NO,team1_NYG,team1_NYJ,team1_OAK,team1_PHI,team1_PIT,team1_SEA,team1_SF,team1_TB,team1_TEN,team1_WSH,team2_ARI,team2_ATL,team2_BAL,team2_BUF,team2_CAR,team2_CHI,team2_CIN,team2_CLE,team2_DAL,team2_DEN,team2_DET,team2_GB,team2_HOU,team2_IND,team2_JAX,team2_KC,team2_LAC,team2_LAR,team2_MIA,team2_MIN,team2_NE,team2_NO,team2_NYG,team2_NYJ,team2_OAK,team2_PHI,team2_PIT,team2_SEA,team2_SF,team2_TB,team2_TEN,team2_WSH
16007,2018,18.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
16008,2018,23.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
16009,2018,24.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
16010,2018,27.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
16011,2018,40.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32873,2018,28.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
32874,2018,14.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
32875,2018,26.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
32876,2018,37.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [11]:
X=ayd2.drop(columns=['season','score1']).to_numpy()
X=torch.from_numpy(X).float()

y=ayd2['score1'].to_numpy()
y=torch.from_numpy(y).float()

print(X[0])
y

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])


tensor([18., 23., 24., 27., 40., 15., 21., 47., 27., 28., 16., 27.,  6., 24.,
        17., 13., 34., 29., 31.,  9., 20., 20., 12., 37., 21., 27., 34., 30.,
        20., 31., 20., 24., 21., 27., 28.,  6.,  6., 38., 31., 22., 20., 31.,
        37., 35., 24., 14., 26., 27., 38., 34., 26., 31., 26., 22., 36., 48.,
        38., 45., 17., 18., 29., 14., 23., 38., 31., 12., 41., 13., 30., 33.,
        27., 34., 26., 18., 31., 21., 19., 43., 13., 20., 14.,  3., 42., 34.,
        23., 31., 21., 27., 20., 40.,  0., 43., 33., 10., 20., 21., 17., 31.,
        26., 37., 17.,  7., 23., 10., 20., 45., 23., 42., 18., 30., 13., 33.,
        37., 14., 36., 24., 28., 29., 18., 20.,  6., 34., 42.,  9., 24., 21.,
        13., 14., 16., 17., 17., 45., 31., 14., 52., 26., 34.,  3., 14., 28.,
        34., 29., 10.,  6., 36., 31., 20., 23., 27., 24., 21., 19., 38., 20.,
        16., 38., 22., 21., 48., 25., 54., 16., 31., 31., 20., 34., 27., 24.,
        27., 25., 13., 45., 27., 24., 24., 34., 13., 21., 17., 2

## attac,def

### simple

In [ ]:
#defino modelo a lo rocho
attack=torch.randn(32,requires_grad=True)
deffense=torch.randn(32,requires_grad=True)
cero=torch.zeros(32)
torch.cat((attack,cero))
def model(x):
  return x @ torch.cat((attack,cero)) - x @ torch.cat((cero,deffense))


In [ ]:
px.scatter(x=list(y.detach().numpy())+list(y.detach().numpy()),
           y=list(model(X).detach().numpy())+list(y.detach().numpy())).show()


In [ ]:
def mse(t1,t2):
  diff = t1 - t2
  return torch.sum(diff * diff) / diff.numel()
  

In [ ]:
#este optimizador parece solido

epochNum=10000
optimizer = torch.optim.Adam([attack, deffense], lr=0.1)

for epoch in range(epochNum):
      
      yhat = model(X)
      loss = mse(yhat, y)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      
       
mse(model(X), y)   

tensor(76.2614, grad_fn=<DivBackward0>)

In [ ]:
mse(model(X), y)   

tensor(76.2614, grad_fn=<DivBackward0>)

In [ ]:
fit(model,[attack, deffense],X,X,y,y,epochNum=5000,lr=0.1,graph=True)

ValueError: ignored

In [ ]:
px.scatter(x=list(y.detach().numpy())+list(y.detach().numpy()),
           y=list(model(X).detach().numpy())+list(y.detach().numpy())).show()



### linear reg included

In [ ]:
#lo defino igual pero con una reg lineal al final
#defino modelo a lo rocho
a=torch.tensor(1.0,requires_grad=True)
b=torch.tensor(1.0,requires_grad=True)
attack=torch.randn(32,requires_grad=True)
deffense=torch.randn(32,requires_grad=True)
cero=torch.zeros(32)
torch.cat((attack,cero))
def model(x):
  return a*(x @ torch.cat((attack,cero)) - x @ torch.cat((cero,deffense)))+b

In [ ]:
#le meto una reg lineal al final
#hago un adam y despues un sgd con scheduler
epochNum=10000
optimizer = torch.optim.Adam([attack, deffense,a,b], lr=0.1)

for epoch in range(epochNum):
      
      yhat = model(X)
      loss = mse(yhat, y)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      

epochNum=10000
optimizer = torch.optim.SGD([attack, deffense,a,b], lr=0.1)
scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.001, max_lr=0.1)
for epoch in range(epochNum):
      
      yhat = model(X)
      loss = mse(yhat, y)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      scheduler.step()        
mse(model(X), y)     

In [ ]:
px.scatter(x=list(y.detach().numpy())+list(y.detach().numpy()),
           y=list(model(X).detach().numpy())+list(y.detach().numpy())).show()


In [ ]:
#deberia ser capaz de hacer una reg lineal al final

### two step

In [ ]:
#defino modelo a lo rocho
attack=torch.randn(32,requires_grad=True)
deffense=torch.randn(32,requires_grad=True)
cero=torch.zeros(32)
torch.cat((attack,cero))
def model(x):
  return x @ torch.cat((attack,cero)) - x @ torch.cat((cero,deffense))


In [ ]:
def mse(t1,t2):
  diff = t1 - t2
  return torch.sum(diff * diff) / diff.numel()
  

In [ ]:
#le meto una reg lineal al final
#hago un adam y despues un sgd con scheduler
epochNum=10000
optimizer = torch.optim.Adam([attack, deffense], lr=0.1)

for epoch in range(epochNum):
      
      yhat = model(X)
      loss = mse(yhat, y)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      

epochNum=10000
optimizer = torch.optim.SGD([attack, deffense], lr=0.1)
scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.001, max_lr=0.1)
for epoch in range(epochNum):
      
      yhat = model(X)
      loss = mse(yhat, y)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      scheduler.step()    
mse(model(X), y)    

In [ ]:
px.scatter(x=list(y.detach().numpy())+list(y.detach().numpy()),
           y=list(model(X).detach().numpy())+list(y.detach().numpy())).show()



In [ ]:
#hago otro modelo
a=torch.tensor(5.0,requires_grad=True)
b=torch.tensor(-(5.0*19),requires_grad=True)
def model1(x):
  return a*model(x)+b
mse(model1(X), y)

In [ ]:
#no deberia ser mejor esa version?
px.scatter(x=list(y.detach().numpy())+list(y.detach().numpy()),
           y=list(model1(X).detach().numpy())+list(y.detach().numpy())).show()
px.scatter(
           x=list(model1(X).detach().numpy())+list(y.detach().numpy()),
           y=list(y.detach().numpy())+list(y.detach().numpy())).show()

In [ ]:
#le meto una reg lineal al final
#hago un adam y despues un sgd con scheduler
epochNum=10000
optimizer = torch.optim.Adam([a, b], lr=0.1)

for epoch in range(epochNum):
      
      yhat = model1(X)
      loss = mse(yhat, y)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      
print(mse(model1(X), y))
# epochNum=1000
# optimizer = torch.optim.SGD([a, b], lr=0.1)
# scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.001, max_lr=0.1)
# for epoch in range(epochNum):
      
#       yhat = model1(X)
#       loss = mse(yhat, y)
#       optimizer.zero_grad()
#       loss.backward()
#       optimizer.step()
#       scheduler.step()    
# mse(model1(X), y)

### chequeo el accuracy

In [ ]:
#tengo que hacer un predict en el elo2

In [ ]:
XX=elo2[['team1','team2']]
XX=pd.get_dummies(XX).to_numpy()
XX=torch.from_numpy(XX).float()
elo2['score1Hat']=model(XX).detach().numpy()
elo2

,team1,team2,elo1_pre,elo2_pre,elo_prob1,y,split,score1Hat,score2Hat,yHat_Ayd
16274,CHI,GB,1588.897931,1455.131370,0.758449,0,1,27.270418,23.539909,1
16275,NYJ,BUF,1384.966442,1448.423682,0.502220,0,0,22.148815,25.225006,0
16276,MIN,ATL,1538.423954,1520.315825,0.617372,1,0,25.128534,27.240158,0
16277,CAR,LAR,1519.378772,1598.015666,0.480385,0,1,23.273487,39.544853,0
16278,JAX,KC,1455.249073,1602.077427,0.384370,0,1,18.782009,34.288696,0
...,...,...,...,...,...,...,...,...,...,...
16536,KC,HOU,1694.572097,1552.346950,0.767256,1,1,33.944675,26.957289,1
16537,GB,SEA,1631.858399,1579.312894,0.662990,1,0,22.359234,34.155361,0
16538,KC,TEN,1707.523977,1662.579362,0.653145,1,1,32.244564,23.213446,1
16539,SF,GB,1651.872498,1643.322690,0.604293,1,0,23.633305,33.275429,0


In [ ]:
#saco el score2
XX=elo2[['team1','team2']]
#tengo que cambiar de lugar eso
elo2Trash=elo2.copy()
XX['team1']=elo2Trash['team2']
XX['team2']=elo2Trash['team1']





XX=pd.get_dummies(XX).to_numpy()
XX=torch.from_numpy(XX).float()
elo2['score2Hat']=model(XX).detach().numpy()
elo2

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,team1,team2,elo1_pre,elo2_pre,elo_prob1,y,split,score1Hat,score2Hat,yHat_Ayd
16274,CHI,GB,1588.897931,1455.131370,0.758449,0,1,27.270418,18.532640,1
16275,NYJ,BUF,1384.966442,1448.423682,0.502220,0,0,22.148815,23.052092,0
16276,MIN,ATL,1538.423954,1520.315825,0.617372,1,0,25.128534,24.193459,0
16277,CAR,LAR,1519.378772,1598.015666,0.480385,0,1,23.273487,30.535137,0
16278,JAX,KC,1455.249073,1602.077427,0.384370,0,1,18.782009,31.694256,0
...,...,...,...,...,...,...,...,...,...,...
16536,KC,HOU,1694.572097,1552.346950,0.767256,1,1,33.944675,28.116219,1
16537,GB,SEA,1631.858399,1579.312894,0.662990,1,0,22.359234,29.075569,0
16538,KC,TEN,1707.523977,1662.579362,0.653145,1,1,32.244564,23.522999,1
16539,SF,GB,1651.872498,1643.322690,0.604293,1,0,23.633305,26.702965,0


In [ ]:
elo2['yHat_Ayd']=0
elo2.loc[elo2['score1Hat']>elo2['score2Hat'],'yHat_Ayd']=1
rateOfSucc(elo2,'yHat_Ayd')

0.6104868913857678

### simple elo combination

In [ ]:
#tengo que hacer un predict en el elo2

In [ ]:
XX=elo2[['team1','team2']]
XX=pd.get_dummies(XX).to_numpy()
XX=torch.from_numpy(XX).float()
elo2['score1Hat']=model(XX).detach().numpy()
elo2

,team1,team2,elo1_pre,elo2_pre,elo_prob1,y,split,score1Hat
16274,CHI,GB,1588.897931,1455.131370,0.758449,0,0,27.270418
16275,NYJ,BUF,1384.966442,1448.423682,0.502220,0,1,22.148817
16276,MIN,ATL,1538.423954,1520.315825,0.617372,1,0,25.128534
16277,CAR,LAR,1519.378772,1598.015666,0.480385,0,1,23.273489
16278,JAX,KC,1455.249073,1602.077427,0.384370,0,1,18.782007
...,...,...,...,...,...,...,...,...
16536,KC,HOU,1694.572097,1552.346950,0.767256,1,1,33.944675
16537,GB,SEA,1631.858399,1579.312894,0.662990,1,1,22.359238
16538,KC,TEN,1707.523977,1662.579362,0.653145,1,0,32.244564
16539,SF,GB,1651.872498,1643.322690,0.604293,1,0,23.633305


In [ ]:
#saco el score2
XX=elo2[['team1','team2']]
#tengo que cambiar de lugar eso
elo2Trash=elo2.copy()
XX['team1']=elo2Trash['team2']
XX['team2']=elo2Trash['team1']





XX=pd.get_dummies(XX).to_numpy()
XX=torch.from_numpy(XX).float()
elo2['score2Hat']=model(XX).detach().numpy()
elo2

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,team1,team2,elo1_pre,elo2_pre,elo_prob1,y,split,score1Hat,score2Hat
16274,CHI,GB,1588.897931,1455.131370,0.758449,0,0,27.270418,18.532642
16275,NYJ,BUF,1384.966442,1448.423682,0.502220,0,1,22.148817,23.052090
16276,MIN,ATL,1538.423954,1520.315825,0.617372,1,0,25.128534,24.193457
16277,CAR,LAR,1519.378772,1598.015666,0.480385,0,1,23.273489,30.535137
16278,JAX,KC,1455.249073,1602.077427,0.384370,0,1,18.782007,31.694256
...,...,...,...,...,...,...,...,...,...
16536,KC,HOU,1694.572097,1552.346950,0.767256,1,1,33.944675,28.116219
16537,GB,SEA,1631.858399,1579.312894,0.662990,1,1,22.359238,29.075569
16538,KC,TEN,1707.523977,1662.579362,0.653145,1,0,32.244564,23.522999
16539,SF,GB,1651.872498,1643.322690,0.604293,1,0,23.633305,26.702965


In [ ]:
elo2['yHat_Ayd']=0
elo2.loc[elo2['score1Hat']>elo2['score2Hat'],'yHat_Ayd']=1
elo2['yHat_elo']=0
elo2.loc[elo2['elo1_pre']>elo2['elo2_pre'],'yHat_elo']=1
rateOfSucc(elo2,'yHat_Ayd')

0.6104868913857678

In [ ]:
#obtengo una funcion que predice la diff en el score dado el elo
anal=elo.loc[elo['season']==2018]
anal['scoreDiff']=anal['score1']-anal['score2']
px.scatter(anal,x='elo_prob1',y='scoreDiff',trendline='ols').show()
elo2['scoreDiffHat_elo']=28*elo2['elo_prob1']-14
elo2

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,team1,team2,elo1_pre,elo2_pre,elo_prob1,y,split,score1Hat,score2Hat,yHat_Ayd,yHat_elo,yHat_elo2,scoreDiffHat_elo
16274,CHI,GB,1588.897931,1455.131370,0.758449,0,0,27.192814,18.512571,1,1,1,7.236559
16275,NYJ,BUF,1384.966442,1448.423682,0.502220,0,1,22.101887,22.975843,0,0,1,0.062166
16276,MIN,ATL,1538.423954,1520.315825,0.617372,1,0,25.072142,24.126812,1,1,1,3.286418
16277,CAR,LAR,1519.378772,1598.015666,0.480385,0,1,23.222229,30.434483,0,0,0,-0.549220
16278,JAX,KC,1455.249073,1602.077427,0.384370,0,1,18.733244,31.547327,0,0,0,-3.237649
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16536,KC,HOU,1694.572097,1552.346950,0.767256,1,1,33.776573,28.021757,1,1,1,7.483180
16537,GB,SEA,1631.858399,1579.312894,0.662990,1,1,22.321535,28.976885,0,1,1,4.563714
16538,KC,TEN,1707.523977,1662.579362,0.653145,1,0,32.091175,23.446938,1,1,1,4.288058
16539,SF,GB,1651.872498,1643.322690,0.604293,1,0,23.568193,26.628872,0,1,1,2.920215


In [ ]:
#combino los 2 modelos
elo2['scoreDiffHat_ayd']=elo2['score1Hat']-elo2['score2Hat']
elo2['scoreDiffHat_simpleComb']=(elo2['scoreDiffHat_ayd']+elo2['scoreDiffHat_elo'])/2
elo2['yHat_simpleComb']=0
elo2.loc[elo2['scoreDiffHat_simpleComb']>0,'yHat_simpleComb']=1
rateOfSucc(elo2,'yHat_simpleComb')

0.6329588014981273

## factor cyclic



In [ ]:
#cambio un poco la forma de laburar
y=ayd2[['score1']].to_numpy()
y=torch.from_numpy(y).float()


In [ ]:
#primero no hago los steps en ciclo, hago todo junto
#2 factores

In [ ]:
#defino modelo a lo rocho
#factores
factors=torch.randn([32,2],requires_grad=True)
cero=torch.zeros([32,2])

#forward
w1=torch.randn([4,3],requires_grad=True)
b1=torch.randn(3,requires_grad=True)
w2=torch.randn([3,3],requires_grad=True)
b2=torch.randn(3,requires_grad=True)
w3=torch.randn([3,1],requires_grad=True)
b3=torch.randn(1,requires_grad=True)

def model(x):
  F1 = x @ torch.cat((factors,cero))
  F2 = x @ torch.cat((cero,factors))
  inputs= torch.cat((F1,F2),axis=1)

  #hago la mini red
  layer1=torch.nn.functional.relu(inputs @ w1 + b1)
  layer2=torch.nn.functional.relu(layer1 @ w2 + b2)
  return torch.nn.functional.relu(layer2 @ w3 + b3)



In [ ]:
px.scatter(x=list(y.detach().numpy().reshape(-1))+list(y.detach().numpy().reshape(-1)),
           y=list(model(X).detach().numpy().reshape(-1))+list(y.detach().numpy().reshape(-1))).show()



In [ ]:
#le meto una reg lineal al final
#hago un adam y despues un sgd con scheduler
epochNum=100
optimizer = torch.optim.Adam([factors, w1,w2,w3,b1,b2,b3], lr=1)

for epoch in range(epochNum):
      
      yhat = model(X)
      loss = mse(yhat, y)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      
       
mse(model(X), y)     

tensor(102.4707, grad_fn=<DivBackward0>)

In [ ]:
px.scatter(x=list(y.detach().numpy().reshape(-1))+list(y.detach().numpy().reshape(-1)),
           y=list(model(X).detach().numpy().reshape(-1))+list(y.detach().numpy().reshape(-1))).show()



In [ ]:
#chequeo la varianza del entrenamiento

In [ ]:
#defino modelo a lo rocho
#factores
factors=torch.randn([32,2],requires_grad=True)
cero=torch.zeros([32,2])

#forward
w1=torch.randn([4,3],requires_grad=True)
b1=torch.randn(3,requires_grad=True)
w2=torch.randn([3,3],requires_grad=True)
b2=torch.randn(3,requires_grad=True)
w3=torch.randn([3,1],requires_grad=True)
b3=torch.randn(1,requires_grad=True)

def model(x):
  F1 = x @ torch.cat((factors,cero))
  F2 = x @ torch.cat((cero,factors))
  inputs= torch.cat((F1,F2),axis=1)

  #hago la mini red
  layer1=torch.nn.functional.relu(inputs @ w1 + b1)
  layer2=torch.nn.functional.relu(layer1 @ w2 + b2)
  return torch.nn.functional.relu(layer2 @ w3 + b3)

epochNum=1000
optimizer = torch.optim.Adam([factors, w1,w2,w3,b1,b2,b3], lr=0.1)

for epoch in range(epochNum):
      
      yhat = model(X)
      loss = mse(yhat, y)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      
       
mse(model(X), y)     


tensor(66.8938, grad_fn=<DivBackward0>)

### quiero forzar el overfitting con los factores

In [ ]:
#defino modelo a lo rocho
qFactors=20
tamaniodered=100
#factores
factors=torch.randn([32,qFactors],requires_grad=True)
cero=torch.zeros([32,qFactors])

#forward
w1=torch.randn([2*qFactors,tamaniodered],requires_grad=True)
b1=torch.randn(tamaniodered,requires_grad=True)
w2=torch.randn([tamaniodered,3],requires_grad=True)
b2=torch.randn(3,requires_grad=True)
w3=torch.randn([3,1],requires_grad=True)
b3=torch.randn(1,requires_grad=True)

def model(x):
  F1 = x @ torch.cat((factors,cero))
  F2 = x @ torch.cat((cero,factors))
  inputs= torch.cat((F1,F2),axis=1)

  #hago la mini red
  layer1=torch.nn.functional.relu(inputs @ w1 + b1)
  layer2=torch.nn.functional.relu(layer1 @ w2 + b2)
  return torch.nn.functional.relu(layer2 @ w3 + b3)

epochNum=10000
optimizer = torch.optim.Adam([factors, w1,w2,w3,b1,b2,b3], lr=0.01,weight_decay=0.8)
losses=[]
for epoch in range(epochNum):
      
      yhat = model(X)
      loss = mse(yhat, y)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      losses.append(loss.detach().numpy())
px.line(x=[t for t in range(epochNum)],y=losses).show()
       
mse(model(X), y)     


tensor(26.3410, grad_fn=<DivBackward0>)

In [ ]:
px.scatter(x=list(y.detach().numpy().reshape(-1))+list(y.detach().numpy().reshape(-1)),
           y=list(model(X).detach().numpy().reshape(-1))+list(y.detach().numpy().reshape(-1))).show()



### asimetric factors

In [ ]:
#defino modelo a lo rocho
qFactors=2
tamaniodered=5
#factores
factors1=torch.randn([32,qFactors],requires_grad=True)
factors2=torch.randn([32,qFactors],requires_grad=True)
cero=torch.zeros([32,qFactors])

#forward
w1=torch.randn([2*qFactors,tamaniodered],requires_grad=True)
b1=torch.randn(tamaniodered,requires_grad=True)
w2=torch.randn([tamaniodered,3],requires_grad=True)
b2=torch.randn(3,requires_grad=True)
w3=torch.randn([3,1],requires_grad=True)
b3=torch.randn(1,requires_grad=True)

def model(x):
  F1 = x @ torch.cat((factors1,cero))
  F2 = x @ torch.cat((cero,factors2))
  inputs= torch.cat((F1,F2),axis=1)

  #hago la mini red
  layer1=torch.nn.functional.relu(inputs @ w1 + b1)
  layer2=torch.nn.functional.relu(layer1 @ w2 + b2)
  return torch.nn.functional.relu(layer2 @ w3 + b3)

epochNum=10000
#weight_decay=1e-10
optimizer = torch.optim.Adam([factors1,factors2, w1,w2,w3,b1,b2,b3], lr=0.1)
losses=[]
for epoch in range(epochNum):
      
      yhat = model(X)
      loss = mse(yhat, y)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      losses.append(loss.detach().numpy())
px.line(x=[t for t in range(epochNum)],y=losses).show()
       
mse(model(X), y)     


tensor(58.7649, grad_fn=<DivBackward0>)

In [ ]:
px.scatter(x=list(y.detach().numpy().reshape(-1))+list(y.detach().numpy().reshape(-1)),
           y=list(model(X).detach().numpy().reshape(-1))+list(y.detach().numpy().reshape(-1))).show()



### dual fire

In [ ]:
#defino modelo a lo rocho
qFactors=10
tamaniodered=20
#factores
factors1=torch.randn([32,qFactors],requires_grad=True)
factors2=torch.randn([32,qFactors],requires_grad=True)
cero=torch.zeros([32,qFactors])

#forward
w1=torch.randn([2*qFactors,tamaniodered],requires_grad=True)
b1=torch.randn(tamaniodered,requires_grad=True)
w2=torch.randn([tamaniodered,3],requires_grad=True)
b2=torch.randn(3,requires_grad=True)
w3=torch.randn([3,1],requires_grad=True)
b3=torch.randn(1,requires_grad=True)

def model(x):
  F1 = x @ torch.cat((factors1,cero))
  F2 = x @ torch.cat((cero,factors2))
  inputs= torch.cat((F1,F2),axis=1)

  #hago la mini red
  layer1=torch.nn.functional.relu(inputs @ w1 + b1)
  layer2=torch.nn.functional.relu(layer1 @ w2 + b2)
  return torch.nn.functional.relu(layer2 @ w3 + b3)

epochNum=5000
#weight_decay=1e-10
optimizer1 = torch.optim.Adam([w1,w2,w3,b1,b2,b3], lr=0.1,weight_decay=5)
optimizer2 = torch.optim.Adam([factors1,factors2], lr=0.1)
losses=[]
for epoch in range(epochNum):

  #primero la red    
  yhat = model(X)
  loss = mse(yhat, y)
  optimizer1.zero_grad()
  loss.backward()
  optimizer1.step()

  #segundo los factores    
  yhat = model(X)
  loss = mse(yhat, y)
  optimizer2.zero_grad()
  loss.backward()
  optimizer2.step()

  losses.append(loss.detach().numpy())
px.line(x=[t for t in range(epochNum)],y=losses).show()
       
mse(model(X), y)     


tensor(76.2660, grad_fn=<DivBackward0>)

### chequeo el accuracy

In [ ]:
#tengo que hacer un predict en el elo2

In [ ]:
XX=elo2[['team1','team2']]
XX=pd.get_dummies(XX).to_numpy()
XX=torch.from_numpy(XX).float()
elo2['score1Hat']=model(XX).detach().numpy().reshape(-1)
elo2

,team1,team2,elo1_pre,elo2_pre,elo_prob1,y,split,score1Hat,score2Hat,yHat_Ayd,yHat_elo,yHat_elo2
16274,CHI,GB,1588.897931,1455.131370,0.758449,0,0,27.192814,18.293797,1,1,1
16275,NYJ,BUF,1384.966442,1448.423682,0.502220,0,1,22.101887,30.114391,0,0,1
16276,MIN,ATL,1538.423954,1520.315825,0.617372,1,0,25.072142,26.806156,1,1,1
16277,CAR,LAR,1519.378772,1598.015666,0.480385,0,1,23.222229,43.246422,0,0,0
16278,JAX,KC,1455.249073,1602.077427,0.384370,0,1,18.733244,28.926796,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
16536,KC,HOU,1694.572097,1552.346950,0.767256,1,1,33.776573,32.669437,0,1,1
16537,GB,SEA,1631.858399,1579.312894,0.662990,1,1,22.321535,25.095081,0,1,1
16538,KC,TEN,1707.523977,1662.579362,0.653145,1,0,32.091175,23.080404,1,1,1
16539,SF,GB,1651.872498,1643.322690,0.604293,1,0,23.568193,29.413206,0,1,1


In [ ]:
#saco el score2
XX=elo2[['team1','team2']]
#tengo que cambiar de lugar eso
elo2Trash=elo2.copy()
XX['team1']=elo2Trash['team2']
XX['team2']=elo2Trash['team1']





XX=pd.get_dummies(XX).to_numpy()
XX=torch.from_numpy(XX).float()
elo2['score2Hat']=model(XX).detach().numpy().reshape(-1)
elo2

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,team1,team2,elo1_pre,elo2_pre,elo_prob1,y,split,score1Hat,score2Hat,yHat_Ayd,yHat_elo,yHat_elo2
16274,CHI,GB,1588.897931,1455.131370,0.758449,0,0,27.192814,18.512571,1,1,1
16275,NYJ,BUF,1384.966442,1448.423682,0.502220,0,1,22.101887,22.975843,0,0,1
16276,MIN,ATL,1538.423954,1520.315825,0.617372,1,0,25.072142,24.126812,1,1,1
16277,CAR,LAR,1519.378772,1598.015666,0.480385,0,1,23.222229,30.434483,0,0,0
16278,JAX,KC,1455.249073,1602.077427,0.384370,0,1,18.733244,31.547327,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
16536,KC,HOU,1694.572097,1552.346950,0.767256,1,1,33.776573,28.021757,0,1,1
16537,GB,SEA,1631.858399,1579.312894,0.662990,1,1,22.321535,28.976885,0,1,1
16538,KC,TEN,1707.523977,1662.579362,0.653145,1,0,32.091175,23.446938,1,1,1
16539,SF,GB,1651.872498,1643.322690,0.604293,1,0,23.568193,26.628872,0,1,1


In [ ]:
elo2['yHat_Ayd']=0
elo2.loc[elo2['score1Hat']>elo2['score2Hat'],'yHat_Ayd']=1
rateOfSucc(elo2,'yHat_Ayd')

0.6104868913857678

## how to use fit

In [ ]:
#how to use

def parametros():
  attack=torch.randn(32,requires_grad=True)
  deffense=torch.randn(32,requires_grad=True)
  parameters=[attack,deffense]
  return parameters

xt=X
xv=X
yt=y
yv=y


def model(x,parameters):
  attack=parameters[0]
  deffense=parameters[1]
  return x @ torch.cat((attack,cero)) - x @ torch.cat((cero,deffense))

fito=stochFit(fit,stochNum=5,epochNum=1000,lr=0.1,graph=True)


mse(model(X,fito['bestParams']),y)

tensor(76.2614)

# hacemos con los datos anteriores posta

## trabajo con el dataset

In [18]:
df1.dropna(inplace=True)
df1['time']=0
alltimeCounter=0
for anio in range(2006,2021):
  df1.loc[df1['season']==anio,'time']=df1.loc[df1['season']==anio,'week']+alltimeCounter
  alltimeCounter=alltimeCounter+df1.loc[df1['season']==anio,'week'].max()
df1

,Home Team,Away Team,season,week,elo1_pre,elo2_pre,elo_prob1,Home Odds Open,y,team_score,opp_score,time
1,Carolina Panthers,Atlanta Falcons,2006.0,1.0,1595.288000,1492.786000,0.723966,0.696970,0,6.0,20.0,1.0
2,Tampa Bay Buccaneers,Baltimore Ravens,2006.0,1.0,1520.605000,1486.653000,0.638674,0.615385,0,0.0,27.0,1.0
3,New England Patriots,Buffalo Bills,2006.0,1.0,1603.944000,1474.302000,0.754072,0.851852,1,19.0,17.0,1.0
4,Green Bay Packers,Chicago Bears,2006.0,1.0,1464.766000,1515.842000,0.520028,0.357143,0,0.0,26.0,1.0
5,Kansas City Chiefs,Cincinnati Bengals,2006.0,1.0,1581.408000,1531.475000,0.659621,0.565217,0,10.0,23.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
3796,Los Angeles Rams,New York Giants,2020.0,4.0,1575.113707,1348.285082,0.842891,0.862069,1,17.0,9.0,237.0
3797,San Francisco 49ers,Philadelphia Eagles,2020.0,4.0,1615.670408,1460.006767,0.780784,0.729927,0,20.0,25.0,237.0
3798,Miami Dolphins,Seattle Seahawks,2020.0,4.0,1435.437716,1605.437417,0.353332,0.281690,0,23.0,31.0,237.0
3799,Green Bay Packers,Atlanta Falcons,2020.0,4.0,1641.976946,1481.164708,0.785815,0.775194,1,30.0,16.0,237.0


In [19]:
df1['scoreDiff']=df1['team_score']-df1['opp_score']
df1

,Home Team,Away Team,season,week,elo1_pre,elo2_pre,elo_prob1,Home Odds Open,y,team_score,opp_score,time,scoreDiff
1,Carolina Panthers,Atlanta Falcons,2006.0,1.0,1595.288000,1492.786000,0.723966,0.696970,0,6.0,20.0,1.0,-14.0
2,Tampa Bay Buccaneers,Baltimore Ravens,2006.0,1.0,1520.605000,1486.653000,0.638674,0.615385,0,0.0,27.0,1.0,-27.0
3,New England Patriots,Buffalo Bills,2006.0,1.0,1603.944000,1474.302000,0.754072,0.851852,1,19.0,17.0,1.0,2.0
4,Green Bay Packers,Chicago Bears,2006.0,1.0,1464.766000,1515.842000,0.520028,0.357143,0,0.0,26.0,1.0,-26.0
5,Kansas City Chiefs,Cincinnati Bengals,2006.0,1.0,1581.408000,1531.475000,0.659621,0.565217,0,10.0,23.0,1.0,-13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3796,Los Angeles Rams,New York Giants,2020.0,4.0,1575.113707,1348.285082,0.842891,0.862069,1,17.0,9.0,237.0,8.0
3797,San Francisco 49ers,Philadelphia Eagles,2020.0,4.0,1615.670408,1460.006767,0.780784,0.729927,0,20.0,25.0,237.0,-5.0
3798,Miami Dolphins,Seattle Seahawks,2020.0,4.0,1435.437716,1605.437417,0.353332,0.281690,0,23.0,31.0,237.0,-8.0
3799,Green Bay Packers,Atlanta Falcons,2020.0,4.0,1641.976946,1481.164708,0.785815,0.775194,1,30.0,16.0,237.0,14.0


## 1factor vs elo

### es parte del modelo esto

In [ ]:
#deberia estar listo
#hago las predicciones para la primera semana de 2019
sem=217
bac=17 #voy una temporada para atras
data=df1.loc[(df1['time']>sem-bac) & (df1['time']<sem)]
data

,Home Team,Away Team,season,week,elo1_pre,elo2_pre,elo_prob1,Home Odds Open,y,team_score,opp_score,time,scoreDiff
3220,Cincinnati Bengals,Baltimore Ravens,2018.0,2.0,1499.240097,1561.692326,0.503666,0.512821,1,34.0,23.0,201.0,11.0
3221,Los Angeles Rams,Arizona Cardinals,2018.0,2.0,1560.388471,1445.574198,0.737902,0.847458,1,34.0,0.0,201.0,34.0
3222,Atlanta Falcons,Carolina Panthers,2018.0,2.0,1587.848622,1567.174456,0.620855,0.699301,1,31.0,24.0,201.0,7.0
3223,New Orleans Saints,Cleveland Browns,2018.0,2.0,1549.163577,1310.717644,0.851546,0.800000,1,21.0,18.0,201.0,3.0
3224,San Francisco 49ers,Detroit Lions,2018.0,2.0,1459.691508,1470.494958,0.577369,0.704225,1,30.0,27.0,201.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3455,Buffalo Bills,Miami Dolphins,2018.0,17.0,1394.128145,1396.275520,0.589478,0.632911,1,42.0,17.0,216.0,25.0
3456,New England Patriots,New York Jets,2018.0,17.0,1633.544618,1331.577006,0.892106,0.884956,1,38.0,3.0,216.0,35.0
3457,Kansas City Chiefs,Oakland Raiders,2018.0,17.0,1649.009776,1362.709331,0.883113,0.892857,1,35.0,3.0,216.0,32.0
3458,Washington Football Team,Philadelphia Eagles,2018.0,17.0,1430.542116,1602.775278,0.350399,0.289855,0,0.0,24.0,216.0,-24.0


In [ ]:
#todavia no trabajo con la cantidad de filas
data=data[['Home Team','Away Team','scoreDiff']]
#duplico la data
data1=data.copy()
data1['Home Team']=data['Away Team']
data1['Away Team']=data['Home Team']
data1['scoreDiff']=-data['scoreDiff']
tecData=pd.concat([data,data1])
tecData=pd.get_dummies(tecData,columns=['Home Team','Away Team'])
tecData

,scoreDiff,Home Team_Arizona Cardinals,Home Team_Atlanta Falcons,Home Team_Baltimore Ravens,Home Team_Buffalo Bills,Home Team_Carolina Panthers,Home Team_Chicago Bears,Home Team_Cincinnati Bengals,Home Team_Cleveland Browns,Home Team_Dallas Cowboys,Home Team_Denver Broncos,Home Team_Detroit Lions,Home Team_Green Bay Packers,Home Team_Houston Texans,Home Team_Indianapolis Colts,Home Team_Jacksonville Jaguars,Home Team_Kansas City Chiefs,Home Team_Los Angeles Chargers,Home Team_Los Angeles Rams,Home Team_Miami Dolphins,Home Team_Minnesota Vikings,Home Team_New England Patriots,Home Team_New Orleans Saints,Home Team_New York Giants,Home Team_New York Jets,Home Team_Oakland Raiders,Home Team_Philadelphia Eagles,Home Team_Pittsburgh Steelers,Home Team_San Francisco 49ers,Home Team_Seattle Seahawks,Home Team_Tampa Bay Buccaneers,Home Team_Tennessee Titans,Home Team_Washington Football Team,Away Team_Arizona Cardinals,Away Team_Atlanta Falcons,Away Team_Baltimore Ravens,Away Team_Buffalo Bills,Away Team_Carolina Panthers,Away Team_Chicago Bears,Away Team_Cincinnati Bengals,Away Team_Cleveland Browns,Away Team_Dallas Cowboys,Away Team_Denver Broncos,Away Team_Detroit Lions,Away Team_Green Bay Packers,Away Team_Houston Texans,Away Team_Indianapolis Colts,Away Team_Jacksonville Jaguars,Away Team_Kansas City Chiefs,Away Team_Los Angeles Chargers,Away Team_Los Angeles Rams,Away Team_Miami Dolphins,Away Team_Minnesota Vikings,Away Team_New England Patriots,Away Team_New Orleans Saints,Away Team_New York Giants,Away Team_New York Jets,Away Team_Oakland Raiders,Away Team_Philadelphia Eagles,Away Team_Pittsburgh Steelers,Away Team_San Francisco 49ers,Away Team_Seattle Seahawks,Away Team_Tampa Bay Buccaneers,Away Team_Tennessee Titans,Away Team_Washington Football Team
3220,11.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3221,34.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3222,7.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3223,3.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3224,3.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3455,-25.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3456,-35.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3457,-32.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3458,24.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [ ]:
X=tecData.drop(columns=['scoreDiff']).to_numpy()
X=torch.from_numpy(X).float()

y=tecData['scoreDiff'].to_numpy()
y=torch.from_numpy(y).float()

print(X[0])
y

tensor([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])


tensor([ 11.,  34.,   7.,   3.,   3.,   3., -12.,  -5., -11.,  -8.,   0.,  11.,
          7.,   1.,   6.,   7.,   4., -21.,  -2.,  10.,  11.,  13.,  14.,   4.,
         12.,  16.,  -6.,  -5.,   8.,  11.,  -3.,  -3.,   7., -12.,  22.,  -1.,
          3.,   2.,  -3.,  31., -15.,  19.,   3.,   2.,  -3.,  38.,  -4.,  14.,
        -10.,  24.,   3.,   3.,  18.,   8.,  16.,  -2.,  10.,  -2.,   2.,  16.,
          1.,  24., -21.,  10., -21.,   7.,   6.,   3.,   8.,  33.,   3., -24.,
         -3.,  -7., -24.,   5.,   3., -35.,  32.,  -4.,  35.,   3.,   3., -11.,
        -13., -29., -20.,  -7.,  -1.,   1.,   3.,  19.,  15.,  15.,   7.,   2.,
        -14., -10.,  14.,  -6.,   3., -14.,   3.,  -7., -19.,  31., -24., -32.,
         15.,  14.,  -2., -16.,  -8.,  10.,   7.,  -7.,  14., -14.,  31.,  12.,
         12., -31.,  -7.,  12.,   3., -14.,  19.,  24., -37.,   5., -13.,  -4.,
          3.,   1.,   3.,  -3.,  -1.,  -2.,   5.,  -2.,  41.,  -4.,   3.,  28.,
          3.,  14.,  -7.,   8.,  35., -1

### principal


In [ ]:
def mse(t1,t2):
  diff = t1 - t2
  return torch.sum(diff * diff) / diff.numel()
  

In [ ]:
factor=torch.randn(32,requires_grad=True)
cero=torch.zeros(32)

def model(x):
  return x @ torch.cat((factor,cero)) - x @ torch.cat((cero,factor))
mse(model(X), y) 

tensor(204.2203, grad_fn=<DivBackward0>)

In [ ]:
px.scatter(x=list(y.detach().numpy())+list(y.detach().numpy()),
           y=list(model(X).detach().numpy())+list(y.detach().numpy())).show()


In [ ]:
#este optimizador parece solido

epochNum=10000
optimizer = torch.optim.Adam([factor], lr=0.1)

for epoch in range(epochNum):
      
      yhat = model(X)
      loss = mse(yhat, y)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      
       
mse(model(X), y)   

tensor(137.0423, grad_fn=<DivBackward0>)

In [ ]:
px.scatter(x=list(y.detach().numpy())+list(y.detach().numpy()),
           y=list(model(X).detach().numpy())+list(y.detach().numpy())).show()



### chequeo el accuracy

In [ ]:
df2=pd.get_dummies(df1.loc[df1['season']==2019],columns=['Home Team','Away Team'])
df2

,season,week,elo1_pre,elo2_pre,elo_prob1,Home Odds Open,y,team_score,opp_score,time,scoreDiff,Home Team_Arizona Cardinals,Home Team_Atlanta Falcons,Home Team_Baltimore Ravens,Home Team_Buffalo Bills,Home Team_Carolina Panthers,Home Team_Chicago Bears,Home Team_Cincinnati Bengals,Home Team_Cleveland Browns,Home Team_Dallas Cowboys,Home Team_Denver Broncos,Home Team_Detroit Lions,Home Team_Green Bay Packers,Home Team_Houston Texans,Home Team_Indianapolis Colts,Home Team_Jacksonville Jaguars,Home Team_Kansas City Chiefs,Home Team_Los Angeles Chargers,Home Team_Los Angeles Rams,Home Team_Miami Dolphins,Home Team_Minnesota Vikings,Home Team_New England Patriots,Home Team_New Orleans Saints,Home Team_New York Giants,Home Team_New York Jets,Home Team_Oakland Raiders,Home Team_Philadelphia Eagles,Home Team_Pittsburgh Steelers,Home Team_San Francisco 49ers,Home Team_Seattle Seahawks,Home Team_Tampa Bay Buccaneers,Home Team_Tennessee Titans,Home Team_Washington Football Team,Away Team_Arizona Cardinals,Away Team_Atlanta Falcons,Away Team_Baltimore Ravens,Away Team_Buffalo Bills,Away Team_Carolina Panthers,Away Team_Chicago Bears,Away Team_Cincinnati Bengals,Away Team_Cleveland Browns,Away Team_Dallas Cowboys,Away Team_Denver Broncos,Away Team_Detroit Lions,Away Team_Green Bay Packers,Away Team_Houston Texans,Away Team_Indianapolis Colts,Away Team_Jacksonville Jaguars,Away Team_Kansas City Chiefs,Away Team_Los Angeles Chargers,Away Team_Los Angeles Rams,Away Team_Miami Dolphins,Away Team_Minnesota Vikings,Away Team_New England Patriots,Away Team_New Orleans Saints,Away Team_New York Giants,Away Team_New York Jets,Away Team_Oakland Raiders,Away Team_Philadelphia Eagles,Away Team_Pittsburgh Steelers,Away Team_San Francisco 49ers,Away Team_Seattle Seahawks,Away Team_Tampa Bay Buccaneers,Away Team_Tennessee Titans,Away Team_Washington Football Team
3471,2019.0,1.0,1588.897931,1455.131370,0.758449,0.632911,0,3.0,10.0,217.0,-7.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3472,2019.0,1.0,1538.423954,1520.315825,0.617372,0.666667,1,28.0,12.0,217.0,16.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3473,2019.0,1.0,1415.178762,1570.539249,0.372816,0.333333,0,10.0,59.0,217.0,-49.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3474,2019.0,1.0,1384.966442,1448.423682,0.502220,0.625000,0,16.0,17.0,217.0,-1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3475,2019.0,1.0,1565.411649,1428.533515,0.761715,0.813008,1,21.0,20.0,217.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3722,2019.0,17.0,1351.254830,1551.327497,0.314850,0.350877,0,17.0,34.0,233.0,-17.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3723,2019.0,17.0,1744.319584,1523.483714,0.838269,0.546448,1,28.0,10.0,233.0,18.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3724,2019.0,17.0,1570.662276,1609.709252,0.537280,0.434783,0,21.0,26.0,233.0,-5.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3725,2019.0,17.0,1584.215388,1542.863113,0.648445,0.357143,0,14.0,35.0,233.0,-21.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [ ]:
#tengo que hacer un predict en el elo2

test=df2.loc[df2['time']==sem]
XX=test[xteam2].to_numpy()
XX=torch.from_numpy(XX).float()
test['scoreDiffHat']=model(XX).detach().numpy()

#rate en primer semana
test['yHat_1factor']=0
test.loc[test['scoreDiffHat']>0,'yHat_1factor']=1
rateOfSucc(test,'yHat_1factor')


0.8125

In [ ]:
test['yHat_elo']=0
test.loc[test['elo1_pre']>test['elo2_pre'],'yHat_elo']=1
rateOfSucc(test,'yHat_elo')

0.8125

### todo adentro

In [ ]:
#deberia estar listo
#hago las predicciones para la primera semana de 2019
testArray=[]
for sem in range(217,234):
  print(sem)
  bac=17 #voy una temporada para atras
  data=df1.loc[(df1['time']>sem-bac) & (df1['time']<sem)]
  


  #todavia no trabajo con la cantidad de filas
  data=data.loc[:,['Home Team','Away Team','scoreDiff']]
  
  #duplico la data
 
  data1=data.copy()
  data1['Home Team']=data.loc[:,'Away Team']
  
  data1['Away Team']=data.loc[:,'Home Team']
  
  data1['scoreDiff']=-data.loc[:,'scoreDiff']
  
  tecData=pd.concat([data,data1])
  tecData=pd.get_dummies(tecData,columns=['Home Team','Away Team'])
  tecData


  X=tecData.drop(columns=['scoreDiff']).to_numpy()
  X=torch.from_numpy(X).float()
  y=tecData.loc[:,'scoreDiff'].to_numpy()
  y=torch.from_numpy(y).float()


  factor=torch.randn(32,requires_grad=True)
  cero=torch.zeros(32)
  def model(x):
    return x @ torch.cat((factor,cero)) - x @ torch.cat((cero,factor))


  epochNum=10000
  optimizer = torch.optim.Adam([factor], lr=0.1)
  for epoch in range(epochNum):  
        yhat = model(X)
        loss = mse(yhat, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
  
  test=df2.loc[df2['time']==sem]
  XX=test.loc[:,xteam2].to_numpy()
  XX=torch.from_numpy(XX).float()
  test['scoreDiffHat']=model(XX).detach().numpy()
  #rate en primer semana
  test['yHat_1factor']=0
  test.loc[test['scoreDiffHat']>0,'yHat_1factor']=1
  print('mine ',rateOfSucc(test,'yHat_1factor'))

  test['yHat_elo']=0
  test.loc[test['elo1_pre']>test['elo2_pre'],'yHat_elo']=1
  testArray.append(test)
  print('elo',rateOfSucc(test,'yHat_elo'))
       
 
finaTest=pd.concat(testArray)
display(finaTest)
print(rateOfSucc(finaTest,'yHat_1factor'))
print(rateOfSucc(finaTest,'yHat_elo'))


217
mine  0.8125
elo 0.8125
218
mine  0.625
elo 0.625
219
mine  0.75
elo 0.6875
220
mine  0.6666666666666666
elo 0.7333333333333333
221
mine  0.4666666666666667
elo 0.5333333333333333
222
mine  0.5714285714285714
elo 0.5
223
mine  0.7142857142857143
elo 0.6428571428571429
224
mine  0.8
elo 0.8666666666666667
225
mine  0.5
elo 0.5
226
mine  0.5384615384615384
elo 0.38461538461538464
227
mine  0.8571428571428571
elo 0.7857142857142857
228
mine  0.7857142857142857
elo 0.7857142857142857
229
mine  0.5625
elo 0.5
230
mine  0.8125
elo 0.6875
231
mine  0.625
elo 0.5625
232
mine  0.5625
elo 0.625
233
mine  0.625
elo 0.625


,season,week,elo1_pre,elo2_pre,elo_prob1,Home Odds Open,y,team_score,opp_score,time,scoreDiff,Home Team_Arizona Cardinals,Home Team_Atlanta Falcons,Home Team_Baltimore Ravens,Home Team_Buffalo Bills,Home Team_Carolina Panthers,Home Team_Chicago Bears,Home Team_Cincinnati Bengals,Home Team_Cleveland Browns,Home Team_Dallas Cowboys,Home Team_Denver Broncos,Home Team_Detroit Lions,Home Team_Green Bay Packers,Home Team_Houston Texans,Home Team_Indianapolis Colts,Home Team_Jacksonville Jaguars,Home Team_Kansas City Chiefs,Home Team_Los Angeles Chargers,Home Team_Los Angeles Rams,Home Team_Miami Dolphins,Home Team_Minnesota Vikings,Home Team_New England Patriots,Home Team_New Orleans Saints,Home Team_New York Giants,Home Team_New York Jets,Home Team_Oakland Raiders,Home Team_Philadelphia Eagles,Home Team_Pittsburgh Steelers,Home Team_San Francisco 49ers,Home Team_Seattle Seahawks,Home Team_Tampa Bay Buccaneers,Home Team_Tennessee Titans,Home Team_Washington Football Team,Away Team_Arizona Cardinals,Away Team_Atlanta Falcons,Away Team_Baltimore Ravens,Away Team_Buffalo Bills,Away Team_Carolina Panthers,Away Team_Chicago Bears,Away Team_Cincinnati Bengals,Away Team_Cleveland Browns,Away Team_Dallas Cowboys,Away Team_Denver Broncos,Away Team_Detroit Lions,Away Team_Green Bay Packers,Away Team_Houston Texans,Away Team_Indianapolis Colts,Away Team_Jacksonville Jaguars,Away Team_Kansas City Chiefs,Away Team_Los Angeles Chargers,Away Team_Los Angeles Rams,Away Team_Miami Dolphins,Away Team_Minnesota Vikings,Away Team_New England Patriots,Away Team_New Orleans Saints,Away Team_New York Giants,Away Team_New York Jets,Away Team_Oakland Raiders,Away Team_Philadelphia Eagles,Away Team_Pittsburgh Steelers,Away Team_San Francisco 49ers,Away Team_Seattle Seahawks,Away Team_Tampa Bay Buccaneers,Away Team_Tennessee Titans,Away Team_Washington Football Team,scoreDiffHat,yHat_1factor,yHat_elo
3471,2019.0,1.0,1588.897931,1455.131370,0.758449,0.632911,0,3.0,10.0,217.0,-7.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10.257267,1,1
3472,2019.0,1.0,1538.423954,1520.315825,0.617372,0.666667,1,28.0,12.0,217.0,16.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.611844,1,1
3473,2019.0,1.0,1415.178762,1570.539249,0.372816,0.333333,0,10.0,59.0,217.0,-49.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-13.854303,0,0
3474,2019.0,1.0,1384.966442,1448.423682,0.502220,0.625000,0,16.0,17.0,217.0,-1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-5.147899,0,0
3475,2019.0,1.0,1565.411649,1428.533515,0.761715,0.813008,1,21.0,20.0,217.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11.198141,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3722,2019.0,17.0,1351.254830,1551.327497,0.314850,0.350877,0,17.0,34.0,233.0,-17.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-6.963134,0,0
3723,2019.0,17.0,1744.319584,1523.483714,0.838269,0.546448,1,28.0,10.0,233.0,18.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,14.993443,1,1
3724,2019.0,17.0,1570.662276,1609.709252,0.537280,0.434783,0,21.0,26.0,233.0,-5.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-8.768667,

0.6640625
0.640625


## ayd vs elo

### es parte del modelo esto

In [ ]:
#deberia estar listo
#hago las predicciones para la primera semana de 2019
sem=217
bac=17 #voy una temporada para atras
data=df1.loc[(df1['time']>sem-bac) & (df1['time']<sem)]
data

,Home Team,Away Team,season,week,elo1_pre,elo2_pre,elo_prob1,Home Odds Open,y,team_score,opp_score,time,scoreDiff
3220,Cincinnati Bengals,Baltimore Ravens,2018.0,2.0,1499.240097,1561.692326,0.503666,0.512821,1,34.0,23.0,201.0,11.0
3221,Los Angeles Rams,Arizona Cardinals,2018.0,2.0,1560.388471,1445.574198,0.737902,0.847458,1,34.0,0.0,201.0,34.0
3222,Atlanta Falcons,Carolina Panthers,2018.0,2.0,1587.848622,1567.174456,0.620855,0.699301,1,31.0,24.0,201.0,7.0
3223,New Orleans Saints,Cleveland Browns,2018.0,2.0,1549.163577,1310.717644,0.851546,0.800000,1,21.0,18.0,201.0,3.0
3224,San Francisco 49ers,Detroit Lions,2018.0,2.0,1459.691508,1470.494958,0.577369,0.704225,1,30.0,27.0,201.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3455,Buffalo Bills,Miami Dolphins,2018.0,17.0,1394.128145,1396.275520,0.589478,0.632911,1,42.0,17.0,216.0,25.0
3456,New England Patriots,New York Jets,2018.0,17.0,1633.544618,1331.577006,0.892106,0.884956,1,38.0,3.0,216.0,35.0
3457,Kansas City Chiefs,Oakland Raiders,2018.0,17.0,1649.009776,1362.709331,0.883113,0.892857,1,35.0,3.0,216.0,32.0
3458,Washington Football Team,Philadelphia Eagles,2018.0,17.0,1430.542116,1602.775278,0.350399,0.289855,0,0.0,24.0,216.0,-24.0


In [ ]:
#todavia no trabajo con la cantidad de filas
data=data[['Home Team','Away Team','team_score','opp_score']]
#duplico la data
data1=data.copy()
data1['Home Team']=data['Away Team']
data1['Away Team']=data['Home Team']
data1['team_score']=data['opp_score']
data1['opp_score']=data['team_score']
tecData=pd.concat([data,data1])
tecData=pd.get_dummies(tecData,columns=['Home Team','Away Team'])
tecData

,team_score,opp_score,Home Team_Arizona Cardinals,Home Team_Atlanta Falcons,Home Team_Baltimore Ravens,Home Team_Buffalo Bills,Home Team_Carolina Panthers,Home Team_Chicago Bears,Home Team_Cincinnati Bengals,Home Team_Cleveland Browns,Home Team_Dallas Cowboys,Home Team_Denver Broncos,Home Team_Detroit Lions,Home Team_Green Bay Packers,Home Team_Houston Texans,Home Team_Indianapolis Colts,Home Team_Jacksonville Jaguars,Home Team_Kansas City Chiefs,Home Team_Los Angeles Chargers,Home Team_Los Angeles Rams,Home Team_Miami Dolphins,Home Team_Minnesota Vikings,Home Team_New England Patriots,Home Team_New Orleans Saints,Home Team_New York Giants,Home Team_New York Jets,Home Team_Oakland Raiders,Home Team_Philadelphia Eagles,Home Team_Pittsburgh Steelers,Home Team_San Francisco 49ers,Home Team_Seattle Seahawks,Home Team_Tampa Bay Buccaneers,Home Team_Tennessee Titans,Home Team_Washington Football Team,Away Team_Arizona Cardinals,Away Team_Atlanta Falcons,Away Team_Baltimore Ravens,Away Team_Buffalo Bills,Away Team_Carolina Panthers,Away Team_Chicago Bears,Away Team_Cincinnati Bengals,Away Team_Cleveland Browns,Away Team_Dallas Cowboys,Away Team_Denver Broncos,Away Team_Detroit Lions,Away Team_Green Bay Packers,Away Team_Houston Texans,Away Team_Indianapolis Colts,Away Team_Jacksonville Jaguars,Away Team_Kansas City Chiefs,Away Team_Los Angeles Chargers,Away Team_Los Angeles Rams,Away Team_Miami Dolphins,Away Team_Minnesota Vikings,Away Team_New England Patriots,Away Team_New Orleans Saints,Away Team_New York Giants,Away Team_New York Jets,Away Team_Oakland Raiders,Away Team_Philadelphia Eagles,Away Team_Pittsburgh Steelers,Away Team_San Francisco 49ers,Away Team_Seattle Seahawks,Away Team_Tampa Bay Buccaneers,Away Team_Tennessee Titans,Away Team_Washington Football Team
3220,34.0,23.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3221,34.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3222,31.0,24.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3223,21.0,18.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3224,30.0,27.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3455,17.0,42.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3456,3.0,38.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3457,3.0,35.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3458,24.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [ ]:
X=tecData.drop(columns=['team_score','opp_score']).to_numpy()
X=torch.from_numpy(X).float()

y=tecData[['team_score','opp_score']].to_numpy()
y=torch.from_numpy(y).float()

print(X[0])
y

tensor([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])


tensor([[34., 23.],
        [34.,  0.],
        [31., 24.],
        [21., 18.],
        [30., 27.],
        [20., 17.],
        [ 9., 21.],
        [37., 42.],
        [20., 31.],
        [12., 20.],
        [29., 29.],
        [31., 20.],
        [20., 13.],
        [20., 19.],
        [27., 21.],
        [24., 17.],
        [21., 17.],
        [ 6., 27.],
        [14., 16.],
        [31., 21.],
        [24., 13.],
        [27., 14.],
        [31., 17.],
        [20., 16.],
        [35., 23.],
        [26., 10.],
        [37., 43.],
        [22., 27.],
        [28., 20.],
        [38., 27.],
        [ 6.,  9.],
        [27., 30.],
        [38., 31.],
        [14., 26.],
        [22.,  0.],
        [36., 37.],
        [45., 42.],
        [26., 24.],
        [34., 37.],
        [38.,  7.],
        [18., 33.],
        [31., 12.],
        [26., 23.],
        [29., 27.],
        [17., 20.],
        [48., 10.],
        [23., 27.],
        [38., 24.],
        [18., 28.],
        [41., 17.],


### principal


In [ ]:
def mse(t1,t2):
  diff = t1 - t2
  return torch.sum(diff * diff) / diff.numel()
  

In [ ]:
attack=torch.randn(32,requires_grad=True)
defense=torch.randn(32,requires_grad=True)
cero=torch.zeros(32)

def model(x):
  score1=x @ torch.cat((attack,cero)) - x @ torch.cat((cero,defense))
  score2=x @ torch.cat((cero,attack)) - x @ torch.cat((defense,cero))
  return torch.stack((score1,score2),axis=1)
mse(model(X), y) 

tensor(645.7400, grad_fn=<DivBackward0>)

In [ ]:
px.scatter(x=list(y[:,0].detach().numpy())+list(y[:,0].detach().numpy()),
           y=list(model(X)[:,0].detach().numpy())+list(y[:,0].detach().numpy())).show()


In [ ]:
#este optimizador parece solido

epochNum=10000
optimizer = torch.optim.Adam([attack,defense], lr=0.1)

for epoch in range(epochNum):
      
      yhat = model(X)
      loss = mse(yhat, y)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      
       
mse(model(X), y)   

tensor(72.9020, grad_fn=<DivBackward0>)

In [ ]:
px.scatter(x=list(y[:,0].detach().numpy())+list(y[:,0].detach().numpy()),
           y=list(model(X)[:,0].detach().numpy())+list(y[:,0].detach().numpy())).show()


### chequeo el accuracy

In [ ]:
df2=pd.get_dummies(df1.loc[df1['season']==2019],columns=['Home Team','Away Team'])
df2

,season,week,elo1_pre,elo2_pre,elo_prob1,Home Odds Open,y,team_score,opp_score,time,scoreDiff,Home Team_Arizona Cardinals,Home Team_Atlanta Falcons,Home Team_Baltimore Ravens,Home Team_Buffalo Bills,Home Team_Carolina Panthers,Home Team_Chicago Bears,Home Team_Cincinnati Bengals,Home Team_Cleveland Browns,Home Team_Dallas Cowboys,Home Team_Denver Broncos,Home Team_Detroit Lions,Home Team_Green Bay Packers,Home Team_Houston Texans,Home Team_Indianapolis Colts,Home Team_Jacksonville Jaguars,Home Team_Kansas City Chiefs,Home Team_Los Angeles Chargers,Home Team_Los Angeles Rams,Home Team_Miami Dolphins,Home Team_Minnesota Vikings,Home Team_New England Patriots,Home Team_New Orleans Saints,Home Team_New York Giants,Home Team_New York Jets,Home Team_Oakland Raiders,Home Team_Philadelphia Eagles,Home Team_Pittsburgh Steelers,Home Team_San Francisco 49ers,Home Team_Seattle Seahawks,Home Team_Tampa Bay Buccaneers,Home Team_Tennessee Titans,Home Team_Washington Football Team,Away Team_Arizona Cardinals,Away Team_Atlanta Falcons,Away Team_Baltimore Ravens,Away Team_Buffalo Bills,Away Team_Carolina Panthers,Away Team_Chicago Bears,Away Team_Cincinnati Bengals,Away Team_Cleveland Browns,Away Team_Dallas Cowboys,Away Team_Denver Broncos,Away Team_Detroit Lions,Away Team_Green Bay Packers,Away Team_Houston Texans,Away Team_Indianapolis Colts,Away Team_Jacksonville Jaguars,Away Team_Kansas City Chiefs,Away Team_Los Angeles Chargers,Away Team_Los Angeles Rams,Away Team_Miami Dolphins,Away Team_Minnesota Vikings,Away Team_New England Patriots,Away Team_New Orleans Saints,Away Team_New York Giants,Away Team_New York Jets,Away Team_Oakland Raiders,Away Team_Philadelphia Eagles,Away Team_Pittsburgh Steelers,Away Team_San Francisco 49ers,Away Team_Seattle Seahawks,Away Team_Tampa Bay Buccaneers,Away Team_Tennessee Titans,Away Team_Washington Football Team
3471,2019.0,1.0,1588.897931,1455.131370,0.758449,0.632911,0,3.0,10.0,217.0,-7.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3472,2019.0,1.0,1538.423954,1520.315825,0.617372,0.666667,1,28.0,12.0,217.0,16.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3473,2019.0,1.0,1415.178762,1570.539249,0.372816,0.333333,0,10.0,59.0,217.0,-49.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3474,2019.0,1.0,1384.966442,1448.423682,0.502220,0.625000,0,16.0,17.0,217.0,-1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3475,2019.0,1.0,1565.411649,1428.533515,0.761715,0.813008,1,21.0,20.0,217.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3722,2019.0,17.0,1351.254830,1551.327497,0.314850,0.350877,0,17.0,34.0,233.0,-17.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3723,2019.0,17.0,1744.319584,1523.483714,0.838269,0.546448,1,28.0,10.0,233.0,18.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3724,2019.0,17.0,1570.662276,1609.709252,0.537280,0.434783,0,21.0,26.0,233.0,-5.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3725,2019.0,17.0,1584.215388,1542.863113,0.648445,0.357143,0,14.0,35.0,233.0,-21.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [ ]:
#tengo que hacer un predict en el elo2

test=df2.loc[df2['time']==sem]
XX=test[xteam2].to_numpy()
XX=torch.from_numpy(XX).float()
test['score1Hat']=model(XX)[:,0].detach().numpy()
test['score2Hat']=model(XX)[:,1].detach().numpy()

#rate en primer semana
test['yHat_1factor']=0
test.loc[test['score1Hat']>test['score2Hat'],'yHat_1factor']=1
rateOfSucc(test,'yHat_1factor')


0.8125

In [ ]:
test['yHat_elo']=0
test.loc[test['elo1_pre']>test['elo2_pre'],'yHat_elo']=1
rateOfSucc(test,'yHat_elo')

0.8125

### todo adentro

In [ ]:
#deberia estar listo
#hago las predicciones para la primera semana de 2019
testArray=[]
for sem in range(217,234):
  print(sem)
  bac=17 #voy una temporada para atras
  data=df1.loc[(df1['time']>sem-bac) & (df1['time']<sem)]


  #todavia no trabajo con la cantidad de filas
  data=data[['Home Team','Away Team','team_score','opp_score']]
  #duplico la data
  data1=data.copy()
  data1['Home Team']=data['Away Team']
  data1['Away Team']=data['Home Team']
  data1['team_score']=data['opp_score']
  data1['opp_score']=data['team_score']
  tecData=pd.concat([data,data1])
  tecData=pd.get_dummies(tecData,columns=['Home Team','Away Team'])

  

  X=tecData.drop(columns=['team_score','opp_score']).to_numpy()
  X=torch.from_numpy(X).float()
  y=tecData[['team_score','opp_score']].to_numpy()
  y=torch.from_numpy(y).float()



  attack=torch.randn(32,requires_grad=True)
  defense=torch.randn(32,requires_grad=True)
  cero=torch.zeros(32)
  def model(x):
    score1=x @ torch.cat((attack,cero)) - x @ torch.cat((cero,defense))
    score2=x @ torch.cat((cero,attack)) - x @ torch.cat((defense,cero))
    return torch.stack((score1,score2),axis=1)


  epochNum=10000
  optimizer = torch.optim.Adam([attack,defense], lr=0.1)
  for epoch in range(epochNum):  
        yhat = model(X)
        loss = mse(yhat, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        

  #tengo que hacer un predict en el elo2
  test=df2.loc[df2['time']==sem]
  XX=test[xteam2].to_numpy()
  XX=torch.from_numpy(XX).float()
  test['score1Hat']=model(XX)[:,0].detach().numpy()
  test['score2Hat']=model(XX)[:,1].detach().numpy()
  #rate en primer semana
  test['yHat_1factor']=0
  test.loc[test['score1Hat']>test['score2Hat'],'yHat_1factor']=1
  print('mine ',rateOfSucc(test,'yHat_1factor'))



  test['yHat_elo']=0
  test.loc[test['elo1_pre']>test['elo2_pre'],'yHat_elo']=1
  testArray.append(test)
  print('elo',rateOfSucc(test,'yHat_elo'))
       
 
finaTest=pd.concat(testArray)
display(finaTest)
print(rateOfSucc(finaTest,'yHat_1factor'))
print(rateOfSucc(finaTest,'yHat_elo'))


217
mine  0.8125
elo 0.8125
218
mine  0.625
elo 0.625
219
mine  0.75
elo 0.6875
220
mine  0.6666666666666666
elo 0.7333333333333333
221
mine  0.4666666666666667
elo 0.5333333333333333
222
mine  0.5714285714285714
elo 0.5
223
mine  0.7142857142857143
elo 0.6428571428571429
224
mine  0.8
elo 0.8666666666666667
225
mine  0.5
elo 0.5
226
mine  0.5384615384615384
elo 0.38461538461538464
227
mine  0.8571428571428571
elo 0.7857142857142857
228
mine  0.7857142857142857
elo 0.7857142857142857
229
mine  0.5625
elo 0.5
230
mine  0.8125
elo 0.6875
231
mine  0.625
elo 0.5625
232
mine  0.5625
elo 0.625
233
mine  0.625
elo 0.625


,season,week,elo1_pre,elo2_pre,elo_prob1,Home Odds Open,y,team_score,opp_score,time,scoreDiff,Home Team_Arizona Cardinals,Home Team_Atlanta Falcons,Home Team_Baltimore Ravens,Home Team_Buffalo Bills,Home Team_Carolina Panthers,Home Team_Chicago Bears,Home Team_Cincinnati Bengals,Home Team_Cleveland Browns,Home Team_Dallas Cowboys,Home Team_Denver Broncos,Home Team_Detroit Lions,Home Team_Green Bay Packers,Home Team_Houston Texans,Home Team_Indianapolis Colts,Home Team_Jacksonville Jaguars,Home Team_Kansas City Chiefs,Home Team_Los Angeles Chargers,Home Team_Los Angeles Rams,Home Team_Miami Dolphins,Home Team_Minnesota Vikings,Home Team_New England Patriots,Home Team_New Orleans Saints,Home Team_New York Giants,Home Team_New York Jets,Home Team_Oakland Raiders,Home Team_Philadelphia Eagles,Home Team_Pittsburgh Steelers,Home Team_San Francisco 49ers,Home Team_Seattle Seahawks,Home Team_Tampa Bay Buccaneers,Home Team_Tennessee Titans,Home Team_Washington Football Team,Away Team_Arizona Cardinals,Away Team_Atlanta Falcons,Away Team_Baltimore Ravens,Away Team_Buffalo Bills,Away Team_Carolina Panthers,Away Team_Chicago Bears,Away Team_Cincinnati Bengals,Away Team_Cleveland Browns,Away Team_Dallas Cowboys,Away Team_Denver Broncos,Away Team_Detroit Lions,Away Team_Green Bay Packers,Away Team_Houston Texans,Away Team_Indianapolis Colts,Away Team_Jacksonville Jaguars,Away Team_Kansas City Chiefs,Away Team_Los Angeles Chargers,Away Team_Los Angeles Rams,Away Team_Miami Dolphins,Away Team_Minnesota Vikings,Away Team_New England Patriots,Away Team_New Orleans Saints,Away Team_New York Giants,Away Team_New York Jets,Away Team_Oakland Raiders,Away Team_Philadelphia Eagles,Away Team_Pittsburgh Steelers,Away Team_San Francisco 49ers,Away Team_Seattle Seahawks,Away Team_Tampa Bay Buccaneers,Away Team_Tennessee Titans,Away Team_Washington Football Team,score1Hat,score2Hat,yHat_1factor,yHat_elo
3471,2019.0,1.0,1588.897931,1455.131370,0.758449,0.632911,0,3.0,10.0,217.0,-7.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,28.388092,18.134600,1,1
3472,2019.0,1.0,1538.423954,1520.315825,0.617372,0.666667,1,28.0,12.0,217.0,16.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,26.400383,24.786400,1,1
3473,2019.0,1.0,1415.178762,1570.539249,0.372816,0.333333,0,10.0,59.0,217.0,-49.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13.727209,27.582188,0,0
3474,2019.0,1.0,1384.966442,1448.423682,0.502220,0.625000,0,16.0,17.0,217.0,-1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,18.880608,24.028740,0,0
3475,2019.0,1.0,1565.411649,1428.533515,0.761715,0.813008,1,21.0,20.0,217.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,30.899139,19.701138,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3722,2019.0,17.0,1351.254830,1551.327497,0.314850,0.350877,0,17.0,34.0,233.0,-17.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,22.207520,29.171093,0,0
3723,2019.0,17.0,1744.319584,1523.483714,0.838269,0.546448,1,28.0,10.0,233.0,18.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,29.475590,14.481887,1,1
3724,2019.0,17.0,1570.662276,1609.709252,0.537280,0.434783,0,21.0,26.0,233.0,-5.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1

0.6640625
0.640625


## abstract factor

### es parte del modelo esto

In [37]:
#deberia estar listo
#hago las predicciones para la primera semana de 2019
sem=217
bac=17 #voy una temporada para atras
data=df1.loc[(df1['time']>sem-bac) & (df1['time']<sem)]
data

,Home Team,Away Team,season,week,elo1_pre,elo2_pre,elo_prob1,Home Odds Open,y,team_score,opp_score,time,scoreDiff
3220,Cincinnati Bengals,Baltimore Ravens,2018.0,2.0,1499.240097,1561.692326,0.503666,0.512821,1,34.0,23.0,201.0,11.0
3221,Los Angeles Rams,Arizona Cardinals,2018.0,2.0,1560.388471,1445.574198,0.737902,0.847458,1,34.0,0.0,201.0,34.0
3222,Atlanta Falcons,Carolina Panthers,2018.0,2.0,1587.848622,1567.174456,0.620855,0.699301,1,31.0,24.0,201.0,7.0
3223,New Orleans Saints,Cleveland Browns,2018.0,2.0,1549.163577,1310.717644,0.851546,0.800000,1,21.0,18.0,201.0,3.0
3224,San Francisco 49ers,Detroit Lions,2018.0,2.0,1459.691508,1470.494958,0.577369,0.704225,1,30.0,27.0,201.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3455,Buffalo Bills,Miami Dolphins,2018.0,17.0,1394.128145,1396.275520,0.589478,0.632911,1,42.0,17.0,216.0,25.0
3456,New England Patriots,New York Jets,2018.0,17.0,1633.544618,1331.577006,0.892106,0.884956,1,38.0,3.0,216.0,35.0
3457,Kansas City Chiefs,Oakland Raiders,2018.0,17.0,1649.009776,1362.709331,0.883113,0.892857,1,35.0,3.0,216.0,32.0
3458,Washington Football Team,Philadelphia Eagles,2018.0,17.0,1430.542116,1602.775278,0.350399,0.289855,0,0.0,24.0,216.0,-24.0


In [38]:
#todavia no trabajo con la cantidad de filas
data=data[['Home Team','Away Team','team_score','opp_score']]
#duplico la data
data1=data.copy()
data1['Home Team']=data['Away Team']
data1['Away Team']=data['Home Team']
data1['team_score']=data['opp_score']
data1['opp_score']=data['team_score']
tecData=pd.concat([data,data1])
tecData=pd.get_dummies(tecData,columns=['Home Team','Away Team'])
tecData

,team_score,opp_score,Home Team_Arizona Cardinals,Home Team_Atlanta Falcons,Home Team_Baltimore Ravens,Home Team_Buffalo Bills,Home Team_Carolina Panthers,Home Team_Chicago Bears,Home Team_Cincinnati Bengals,Home Team_Cleveland Browns,Home Team_Dallas Cowboys,Home Team_Denver Broncos,Home Team_Detroit Lions,Home Team_Green Bay Packers,Home Team_Houston Texans,Home Team_Indianapolis Colts,Home Team_Jacksonville Jaguars,Home Team_Kansas City Chiefs,Home Team_Los Angeles Chargers,Home Team_Los Angeles Rams,Home Team_Miami Dolphins,Home Team_Minnesota Vikings,Home Team_New England Patriots,Home Team_New Orleans Saints,Home Team_New York Giants,Home Team_New York Jets,Home Team_Oakland Raiders,Home Team_Philadelphia Eagles,Home Team_Pittsburgh Steelers,Home Team_San Francisco 49ers,Home Team_Seattle Seahawks,Home Team_Tampa Bay Buccaneers,Home Team_Tennessee Titans,Home Team_Washington Football Team,Away Team_Arizona Cardinals,Away Team_Atlanta Falcons,Away Team_Baltimore Ravens,Away Team_Buffalo Bills,Away Team_Carolina Panthers,Away Team_Chicago Bears,Away Team_Cincinnati Bengals,Away Team_Cleveland Browns,Away Team_Dallas Cowboys,Away Team_Denver Broncos,Away Team_Detroit Lions,Away Team_Green Bay Packers,Away Team_Houston Texans,Away Team_Indianapolis Colts,Away Team_Jacksonville Jaguars,Away Team_Kansas City Chiefs,Away Team_Los Angeles Chargers,Away Team_Los Angeles Rams,Away Team_Miami Dolphins,Away Team_Minnesota Vikings,Away Team_New England Patriots,Away Team_New Orleans Saints,Away Team_New York Giants,Away Team_New York Jets,Away Team_Oakland Raiders,Away Team_Philadelphia Eagles,Away Team_Pittsburgh Steelers,Away Team_San Francisco 49ers,Away Team_Seattle Seahawks,Away Team_Tampa Bay Buccaneers,Away Team_Tennessee Titans,Away Team_Washington Football Team
3220,34.0,23.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3221,34.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3222,31.0,24.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3223,21.0,18.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3224,30.0,27.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3455,17.0,42.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3456,3.0,38.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3457,3.0,35.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3458,24.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [39]:
X=tecData.drop(columns=['team_score','opp_score']).to_numpy()
X=torch.from_numpy(X).float()

y=tecData[['team_score','opp_score']].to_numpy()
y=torch.from_numpy(y).float()

print(X[0])
y

tensor([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])


tensor([[34., 23.],
        [34.,  0.],
        [31., 24.],
        [21., 18.],
        [30., 27.],
        [20., 17.],
        [ 9., 21.],
        [37., 42.],
        [20., 31.],
        [12., 20.],
        [29., 29.],
        [31., 20.],
        [20., 13.],
        [20., 19.],
        [27., 21.],
        [24., 17.],
        [21., 17.],
        [ 6., 27.],
        [14., 16.],
        [31., 21.],
        [24., 13.],
        [27., 14.],
        [31., 17.],
        [20., 16.],
        [35., 23.],
        [26., 10.],
        [37., 43.],
        [22., 27.],
        [28., 20.],
        [38., 27.],
        [ 6.,  9.],
        [27., 30.],
        [38., 31.],
        [14., 26.],
        [22.,  0.],
        [36., 37.],
        [45., 42.],
        [26., 24.],
        [34., 37.],
        [38.,  7.],
        [18., 33.],
        [31., 12.],
        [26., 23.],
        [29., 27.],
        [17., 20.],
        [48., 10.],
        [23., 27.],
        [38., 24.],
        [18., 28.],
        [41., 17.],


### principal


In [40]:
def mse(t1,t2):
  diff = t1 - t2
  return torch.sum(diff * diff) / diff.numel()
  

In [41]:
#defino modelo a lo rocho
qFactors=5
tamaniodered=10
#factores
factors=torch.randn([32,qFactors],requires_grad=True)
cero=torch.zeros([32,qFactors])

#forward
w1=torch.randn([2*qFactors,tamaniodered],requires_grad=True)
b1=torch.randn(tamaniodered,requires_grad=True)
w2=torch.randn([tamaniodered,tamaniodered],requires_grad=True)
b2=torch.randn(tamaniodered,requires_grad=True)
w3=torch.randn([tamaniodered,2],requires_grad=True)
b3=torch.randn(2,requires_grad=True)

def model(x):
  F1 = x @ torch.cat((factors,cero))
  F2 = x @ torch.cat((cero,factors))
  inputs= torch.cat((F1,F2),axis=1)

  #hago la mini red
  layer1=torch.nn.functional.relu(inputs @ w1 + b1)
  layer2=torch.nn.functional.relu(layer1 @ w2 + b2)
  return torch.nn.functional.relu(layer2 @ w3 + b3)
print(mse(model(X), y))

epochNum=10000
optimizer = torch.optim.Adam([factors,w1,w2,w3,b1,b2,b3], lr=0.1)

for epoch in range(epochNum):
      
      yhat = model(X)
      loss = mse(yhat, y)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      
       
mse(model(X), y)   

tensor(547.9055, grad_fn=<DivBackward0>)


tensor(32.6239, grad_fn=<DivBackward0>)

In [42]:
px.scatter(x=list(y[:,0].detach().numpy())+list(y[:,0].detach().numpy()),
           y=list(model(X)[:,0].detach().numpy())+list(y[:,0].detach().numpy())).show()


### chequeo el accuracy

In [43]:
df2=pd.get_dummies(df1.loc[df1['season']==2019],columns=['Home Team','Away Team'])
df2

,season,week,elo1_pre,elo2_pre,elo_prob1,Home Odds Open,y,team_score,opp_score,time,scoreDiff,Home Team_Arizona Cardinals,Home Team_Atlanta Falcons,Home Team_Baltimore Ravens,Home Team_Buffalo Bills,Home Team_Carolina Panthers,Home Team_Chicago Bears,Home Team_Cincinnati Bengals,Home Team_Cleveland Browns,Home Team_Dallas Cowboys,Home Team_Denver Broncos,Home Team_Detroit Lions,Home Team_Green Bay Packers,Home Team_Houston Texans,Home Team_Indianapolis Colts,Home Team_Jacksonville Jaguars,Home Team_Kansas City Chiefs,Home Team_Los Angeles Chargers,Home Team_Los Angeles Rams,Home Team_Miami Dolphins,Home Team_Minnesota Vikings,Home Team_New England Patriots,Home Team_New Orleans Saints,Home Team_New York Giants,Home Team_New York Jets,Home Team_Oakland Raiders,Home Team_Philadelphia Eagles,Home Team_Pittsburgh Steelers,Home Team_San Francisco 49ers,Home Team_Seattle Seahawks,Home Team_Tampa Bay Buccaneers,Home Team_Tennessee Titans,Home Team_Washington Football Team,Away Team_Arizona Cardinals,Away Team_Atlanta Falcons,Away Team_Baltimore Ravens,Away Team_Buffalo Bills,Away Team_Carolina Panthers,Away Team_Chicago Bears,Away Team_Cincinnati Bengals,Away Team_Cleveland Browns,Away Team_Dallas Cowboys,Away Team_Denver Broncos,Away Team_Detroit Lions,Away Team_Green Bay Packers,Away Team_Houston Texans,Away Team_Indianapolis Colts,Away Team_Jacksonville Jaguars,Away Team_Kansas City Chiefs,Away Team_Los Angeles Chargers,Away Team_Los Angeles Rams,Away Team_Miami Dolphins,Away Team_Minnesota Vikings,Away Team_New England Patriots,Away Team_New Orleans Saints,Away Team_New York Giants,Away Team_New York Jets,Away Team_Oakland Raiders,Away Team_Philadelphia Eagles,Away Team_Pittsburgh Steelers,Away Team_San Francisco 49ers,Away Team_Seattle Seahawks,Away Team_Tampa Bay Buccaneers,Away Team_Tennessee Titans,Away Team_Washington Football Team
3471,2019.0,1.0,1588.897931,1455.131370,0.758449,0.632911,0,3.0,10.0,217.0,-7.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3472,2019.0,1.0,1538.423954,1520.315825,0.617372,0.666667,1,28.0,12.0,217.0,16.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3473,2019.0,1.0,1415.178762,1570.539249,0.372816,0.333333,0,10.0,59.0,217.0,-49.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3474,2019.0,1.0,1384.966442,1448.423682,0.502220,0.625000,0,16.0,17.0,217.0,-1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3475,2019.0,1.0,1565.411649,1428.533515,0.761715,0.813008,1,21.0,20.0,217.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3722,2019.0,17.0,1351.254830,1551.327497,0.314850,0.350877,0,17.0,34.0,233.0,-17.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3723,2019.0,17.0,1744.319584,1523.483714,0.838269,0.546448,1,28.0,10.0,233.0,18.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3724,2019.0,17.0,1570.662276,1609.709252,0.537280,0.434783,0,21.0,26.0,233.0,-5.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3725,2019.0,17.0,1584.215388,1542.863113,0.648445,0.357143,0,14.0,35.0,233.0,-21.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [44]:
#tengo que hacer un predict en el elo2

test=df2.loc[df2['time']==sem]
XX=test[xteam2].to_numpy()
XX=torch.from_numpy(XX).float()
test['score1Hat']=model(XX)[:,0].detach().numpy()
test['score2Hat']=model(XX)[:,1].detach().numpy()

#rate en primer semana
test['yHat_1factor']=0
test.loc[test['score1Hat']>test['score2Hat'],'yHat_1factor']=1
rateOfSucc(test,'yHat_1factor')


0.625

In [45]:
test['yHat_elo']=0
test.loc[test['elo1_pre']>test['elo2_pre'],'yHat_elo']=1
rateOfSucc(test,'yHat_elo')

0.8125

### todo adentro

In [ ]:
for i in range(100):
  print('-----------------------------')
  sem=217
  bac=17 #voy una temporada para atras
  data=df1.loc[(df1['time']>sem-bac) & (df1['time']<sem)]


  #todavia no trabajo con la cantidad de filas
  data=data[['Home Team','Away Team','team_score','opp_score']]
  #duplico la data
  data1=data.copy()
  data1['Home Team']=data['Away Team']
  data1['Away Team']=data['Home Team']
  data1['team_score']=data['opp_score']
  data1['opp_score']=data['team_score']
  tecData=pd.concat([data,data1])
  tecData=pd.get_dummies(tecData,columns=['Home Team','Away Team'])

    

  X=tecData.drop(columns=['team_score','opp_score']).to_numpy()
  X=torch.from_numpy(X).float()
  y=tecData[['team_score','opp_score']].to_numpy()
  y=torch.from_numpy(y).float()



      #defino modelo a lo rocho
  qFactors=10
  tamaniodered=10
  #factores
  factors=torch.randn([32,qFactors],requires_grad=True)
  cero=torch.zeros([32,qFactors])

  #forward
  w1=torch.randn([2*qFactors,tamaniodered],requires_grad=True)
  b1=torch.randn(tamaniodered,requires_grad=True)
  w2=torch.randn([tamaniodered,tamaniodered],requires_grad=True)
  b2=torch.randn(tamaniodered,requires_grad=True)
  w3=torch.randn([tamaniodered,2],requires_grad=True)
  b3=torch.randn(2,requires_grad=True)

  def model(x):
    F1 = x @ torch.cat((factors,cero))
    F2 = x @ torch.cat((cero,factors))
    inputs= torch.cat((F1,F2),axis=1)

    #hago la mini red
    layer1=torch.nn.functional.relu(inputs @ w1 + b1)
    layer2=torch.nn.functional.relu(layer1 @ w2 + b2)
    return torch.nn.functional.relu(layer2 @ w3 + b3)

  epochNum=5000
  #weight_decay=1e-10
  optimizer1 = torch.optim.Adam([w1,w2,w3,b1,b2,b3], lr=0.1,weight_decay=2)
  optimizer2 = torch.optim.Adam([factors], lr=0.1)
  losses=[]
  for epoch in range(epochNum):

    #primero la red    
    yhat = model(X)
    loss = mse(yhat, y)
    optimizer1.zero_grad()
    loss.backward()
    optimizer1.step()

    #segundo los factores    
    yhat = model(X)
    loss = mse(yhat, y)
    optimizer2.zero_grad()
    loss.backward()
    optimizer2.step()

    losses.append(loss.detach().numpy())
          
  print(mse(model(X), y))
  #tengo que hacer un predict en el elo2
  test=df2.loc[df2['time']==sem]
  XX=test[xteam2].to_numpy()
  XX=torch.from_numpy(XX).float()
  test['score1Hat']=model(XX)[:,0].detach().numpy()
  test['score2Hat']=model(XX)[:,1].detach().numpy()
  #rate en primer semana
  test['yHat_1factor']=0
  test.loc[test['score1Hat']>test['score2Hat'],'yHat_1factor']=1
  print('mine ',rateOfSucc(test,'yHat_1factor'))



  test['yHat_elo']=0
  test.loc[test['elo1_pre']>test['elo2_pre'],'yHat_elo']=1
  testArray.append(test)
  print('elo',rateOfSucc(test,'yHat_elo'))

-----------------------------
tensor(28.5743, grad_fn=<DivBackward0>)
mine  0.625
elo 0.8125
-----------------------------
tensor(29.9914, grad_fn=<DivBackward0>)
mine  0.5625
elo 0.8125
-----------------------------
tensor(28.6231, grad_fn=<DivBackward0>)
mine  0.6875
elo 0.8125
-----------------------------
tensor(337.5932, grad_fn=<DivBackward0>)
mine  0.5
elo 0.8125
-----------------------------
tensor(25.8335, grad_fn=<DivBackward0>)
mine  0.8125
elo 0.8125
-----------------------------
tensor(24.1691, grad_fn=<DivBackward0>)
mine  0.6875
elo 0.8125
-----------------------------
tensor(26.5309, grad_fn=<DivBackward0>)
mine  0.75
elo 0.8125
-----------------------------
tensor(27.6894, grad_fn=<DivBackward0>)
mine  0.6875
elo 0.8125
-----------------------------
tensor(27.0347, grad_fn=<DivBackward0>)
mine  0.625
elo 0.8125
-----------------------------
tensor(26.5816, grad_fn=<DivBackward0>)
mine  0.75
elo 0.8125
-----------------------------
tensor(27.6166, grad_fn=<DivBackward0>

In [46]:
#deberia estar listo
#hago las predicciones para la primera semana de 2019
testArray=[]
for sem in range(217,234):
  print(sem)
  bac=17 #voy una temporada para atras
  data=df1.loc[(df1['time']>sem-bac) & (df1['time']<sem)]


  #todavia no trabajo con la cantidad de filas
  data=data[['Home Team','Away Team','team_score','opp_score']]
  #duplico la data
  data1=data.copy()
  data1['Home Team']=data['Away Team']
  data1['Away Team']=data['Home Team']
  data1['team_score']=data['opp_score']
  data1['opp_score']=data['team_score']
  tecData=pd.concat([data,data1])
  tecData=pd.get_dummies(tecData,columns=['Home Team','Away Team'])

  

  X=tecData.drop(columns=['team_score','opp_score']).to_numpy()
  X=torch.from_numpy(X).float()
  y=tecData[['team_score','opp_score']].to_numpy()
  y=torch.from_numpy(y).float()



    #defino modelo a lo rocho
  qFactors=5
  tamaniodered=10
  #factores
  factors=torch.randn([32,qFactors],requires_grad=True)
  cero=torch.zeros([32,qFactors])

  #forward
  w1=torch.randn([2*qFactors,tamaniodered],requires_grad=True)
  b1=torch.randn(tamaniodered,requires_grad=True)
  w2=torch.randn([tamaniodered,tamaniodered],requires_grad=True)
  b2=torch.randn(tamaniodered,requires_grad=True)
  w3=torch.randn([tamaniodered,2],requires_grad=True)
  b3=torch.randn(2,requires_grad=True)

  def model(x):
    F1 = x @ torch.cat((factors,cero))
    F2 = x @ torch.cat((cero,factors))
    inputs= torch.cat((F1,F2),axis=1)

    #hago la mini red
    layer1=torch.nn.functional.relu(inputs @ w1 + b1)
    layer2=torch.nn.functional.relu(layer1 @ w2 + b2)
    return torch.nn.functional.relu(layer2 @ w3 + b3)

  epochNum=10000
  optimizer = torch.optim.Adam([factors,w1,w2,w3,b1,b2,b3], lr=0.1)
  for epoch in range(epochNum):  
        yhat = model(X)
        loss = mse(yhat, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        

  #tengo que hacer un predict en el elo2
  test=df2.loc[df2['time']==sem]
  XX=test[xteam2].to_numpy()
  XX=torch.from_numpy(XX).float()
  test['score1Hat']=model(XX)[:,0].detach().numpy()
  test['score2Hat']=model(XX)[:,1].detach().numpy()
  #rate en primer semana
  test['yHat_1factor']=0
  test.loc[test['score1Hat']>test['score2Hat'],'yHat_1factor']=1
  print('mine ',rateOfSucc(test,'yHat_1factor'))



  test['yHat_elo']=0
  test.loc[test['elo1_pre']>test['elo2_pre'],'yHat_elo']=1
  testArray.append(test)
  print('elo',rateOfSucc(test,'yHat_elo'))
       
 
finaTest=pd.concat(testArray)
display(finaTest)
print(rateOfSucc(finaTest,'yHat_1factor'))
print(rateOfSucc(finaTest,'yHat_elo'))


217
mine  0.6875
elo 0.8125
218
mine  0.6875
elo 0.625
219
mine  0.8125
elo 0.6875
220
mine  0.5333333333333333
elo 0.7333333333333333
221
mine  0.4
elo 0.5333333333333333
222
mine  0.42857142857142855
elo 0.5
223
mine  0.7142857142857143
elo 0.6428571428571429
224
mine  0.5333333333333333
elo 0.8666666666666667
225
mine  0.5714285714285714
elo 0.5
226
mine  0.46153846153846156
elo 0.38461538461538464
227
mine  0.5714285714285714
elo 0.7857142857142857
228
mine  0.5
elo 0.7857142857142857
229
mine  0.4375
elo 0.5
230
mine  0.5
elo 0.6875
231
mine  0.75
elo 0.5625
232
mine  0.3125
elo 0.625
233
mine  0.4375
elo 0.625


,season,week,elo1_pre,elo2_pre,elo_prob1,Home Odds Open,y,team_score,opp_score,time,scoreDiff,Home Team_Arizona Cardinals,Home Team_Atlanta Falcons,Home Team_Baltimore Ravens,Home Team_Buffalo Bills,Home Team_Carolina Panthers,Home Team_Chicago Bears,Home Team_Cincinnati Bengals,Home Team_Cleveland Browns,Home Team_Dallas Cowboys,Home Team_Denver Broncos,Home Team_Detroit Lions,Home Team_Green Bay Packers,Home Team_Houston Texans,Home Team_Indianapolis Colts,Home Team_Jacksonville Jaguars,Home Team_Kansas City Chiefs,Home Team_Los Angeles Chargers,Home Team_Los Angeles Rams,Home Team_Miami Dolphins,Home Team_Minnesota Vikings,Home Team_New England Patriots,Home Team_New Orleans Saints,Home Team_New York Giants,Home Team_New York Jets,Home Team_Oakland Raiders,Home Team_Philadelphia Eagles,Home Team_Pittsburgh Steelers,Home Team_San Francisco 49ers,Home Team_Seattle Seahawks,Home Team_Tampa Bay Buccaneers,Home Team_Tennessee Titans,Home Team_Washington Football Team,Away Team_Arizona Cardinals,Away Team_Atlanta Falcons,Away Team_Baltimore Ravens,Away Team_Buffalo Bills,Away Team_Carolina Panthers,Away Team_Chicago Bears,Away Team_Cincinnati Bengals,Away Team_Cleveland Browns,Away Team_Dallas Cowboys,Away Team_Denver Broncos,Away Team_Detroit Lions,Away Team_Green Bay Packers,Away Team_Houston Texans,Away Team_Indianapolis Colts,Away Team_Jacksonville Jaguars,Away Team_Kansas City Chiefs,Away Team_Los Angeles Chargers,Away Team_Los Angeles Rams,Away Team_Miami Dolphins,Away Team_Minnesota Vikings,Away Team_New England Patriots,Away Team_New Orleans Saints,Away Team_New York Giants,Away Team_New York Jets,Away Team_Oakland Raiders,Away Team_Philadelphia Eagles,Away Team_Pittsburgh Steelers,Away Team_San Francisco 49ers,Away Team_Seattle Seahawks,Away Team_Tampa Bay Buccaneers,Away Team_Tennessee Titans,Away Team_Washington Football Team,score1Hat,score2Hat,yHat_1factor,yHat_elo
3471,2019.0,1.0,1588.897931,1455.131370,0.758449,0.632911,0,3.0,10.0,217.0,-7.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,29.261337,28.114017,1,1
3472,2019.0,1.0,1538.423954,1520.315825,0.617372,0.666667,1,28.0,12.0,217.0,16.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,30.193459,16.048615,1,1
3473,2019.0,1.0,1415.178762,1570.539249,0.372816,0.333333,0,10.0,59.0,217.0,-49.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,24.619728,28.128643,0,0
3474,2019.0,1.0,1384.966442,1448.423682,0.502220,0.625000,0,16.0,17.0,217.0,-1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,18.184261,32.492447,0,0
3475,2019.0,1.0,1565.411649,1428.533515,0.761715,0.813008,1,21.0,20.0,217.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,20.850372,28.361614,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3722,2019.0,17.0,1351.254830,1551.327497,0.314850,0.350877,0,17.0,34.0,233.0,-17.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,26.459589,23.752411,1,0
3723,2019.0,17.0,1744.319584,1523.483714,0.838269,0.546448,1,28.0,10.0,233.0,18.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,27.336582,20.207235,1,1
3724,2019.0,17.0,1570.662276,1609.709252,0.537280,0.434783,0,21.0,26.0,233.0,-5.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1

0.55078125
0.640625


## no volatility

### es parte del modelo esto

In [23]:
#deberia estar listo
#hago las predicciones para la primera semana de 2019
sem=217
bac=17 #voy una temporada para atras
data=df1.loc[(df1['time']>sem-bac) & (df1['time']<sem)]
data

,Home Team,Away Team,season,week,elo1_pre,elo2_pre,elo_prob1,Home Odds Open,y,team_score,opp_score,time,scoreDiff
3220,Cincinnati Bengals,Baltimore Ravens,2018.0,2.0,1499.240097,1561.692326,0.503666,0.512821,1,34.0,23.0,201.0,11.0
3221,Los Angeles Rams,Arizona Cardinals,2018.0,2.0,1560.388471,1445.574198,0.737902,0.847458,1,34.0,0.0,201.0,34.0
3222,Atlanta Falcons,Carolina Panthers,2018.0,2.0,1587.848622,1567.174456,0.620855,0.699301,1,31.0,24.0,201.0,7.0
3223,New Orleans Saints,Cleveland Browns,2018.0,2.0,1549.163577,1310.717644,0.851546,0.800000,1,21.0,18.0,201.0,3.0
3224,San Francisco 49ers,Detroit Lions,2018.0,2.0,1459.691508,1470.494958,0.577369,0.704225,1,30.0,27.0,201.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3455,Buffalo Bills,Miami Dolphins,2018.0,17.0,1394.128145,1396.275520,0.589478,0.632911,1,42.0,17.0,216.0,25.0
3456,New England Patriots,New York Jets,2018.0,17.0,1633.544618,1331.577006,0.892106,0.884956,1,38.0,3.0,216.0,35.0
3457,Kansas City Chiefs,Oakland Raiders,2018.0,17.0,1649.009776,1362.709331,0.883113,0.892857,1,35.0,3.0,216.0,32.0
3458,Washington Football Team,Philadelphia Eagles,2018.0,17.0,1430.542116,1602.775278,0.350399,0.289855,0,0.0,24.0,216.0,-24.0


In [24]:
data['split']=np.random.binomial(1, 0.8, len(data))
data1=data.copy()
data1['Home Team']=data['Away Team']
data1['Away Team']=data['Home Team']
data1['team_score']=data['opp_score']
data1['opp_score']=data['team_score']
data=pd.concat([data,data1])
tecData=pd.get_dummies(data,columns=['Home Team','Away Team'])
trainData=tecData.loc[tecData['split']==1]
valData=tecData.loc[tecData['split']==0]



In [25]:
#todavia no trabajo con la cantidad de filas
trainData=trainData[xteam2+['team_score','opp_score']]
#duplico la trainData

trainData

,Home Team_Arizona Cardinals,Home Team_Atlanta Falcons,Home Team_Baltimore Ravens,Home Team_Buffalo Bills,Home Team_Carolina Panthers,Home Team_Chicago Bears,Home Team_Cincinnati Bengals,Home Team_Cleveland Browns,Home Team_Dallas Cowboys,Home Team_Denver Broncos,Home Team_Detroit Lions,Home Team_Green Bay Packers,Home Team_Houston Texans,Home Team_Indianapolis Colts,Home Team_Jacksonville Jaguars,Home Team_Kansas City Chiefs,Home Team_Los Angeles Chargers,Home Team_Los Angeles Rams,Home Team_Miami Dolphins,Home Team_Minnesota Vikings,Home Team_New England Patriots,Home Team_New Orleans Saints,Home Team_New York Giants,Home Team_New York Jets,Home Team_Oakland Raiders,Home Team_Philadelphia Eagles,Home Team_Pittsburgh Steelers,Home Team_San Francisco 49ers,Home Team_Seattle Seahawks,Home Team_Tampa Bay Buccaneers,Home Team_Tennessee Titans,Home Team_Washington Football Team,Away Team_Arizona Cardinals,Away Team_Atlanta Falcons,Away Team_Baltimore Ravens,Away Team_Buffalo Bills,Away Team_Carolina Panthers,Away Team_Chicago Bears,Away Team_Cincinnati Bengals,Away Team_Cleveland Browns,Away Team_Dallas Cowboys,Away Team_Denver Broncos,Away Team_Detroit Lions,Away Team_Green Bay Packers,Away Team_Houston Texans,Away Team_Indianapolis Colts,Away Team_Jacksonville Jaguars,Away Team_Kansas City Chiefs,Away Team_Los Angeles Chargers,Away Team_Los Angeles Rams,Away Team_Miami Dolphins,Away Team_Minnesota Vikings,Away Team_New England Patriots,Away Team_New Orleans Saints,Away Team_New York Giants,Away Team_New York Jets,Away Team_Oakland Raiders,Away Team_Philadelphia Eagles,Away Team_Pittsburgh Steelers,Away Team_San Francisco 49ers,Away Team_Seattle Seahawks,Away Team_Tampa Bay Buccaneers,Away Team_Tennessee Titans,Away Team_Washington Football Team,team_score,opp_score
3220,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,34.0,23.0
3221,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,34.0,0.0
3222,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,31.0,24.0
3223,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,21.0,18.0
3224,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,30.0,27.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3455,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,17.0,42.0
3456,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,3.0,38.0
3457,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.0,35.0
3458,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,24.0,0.0


In [26]:
valData=valData[xteam2+['team_score','opp_score']]
#duplico la valData
valData

,Home Team_Arizona Cardinals,Home Team_Atlanta Falcons,Home Team_Baltimore Ravens,Home Team_Buffalo Bills,Home Team_Carolina Panthers,Home Team_Chicago Bears,Home Team_Cincinnati Bengals,Home Team_Cleveland Browns,Home Team_Dallas Cowboys,Home Team_Denver Broncos,Home Team_Detroit Lions,Home Team_Green Bay Packers,Home Team_Houston Texans,Home Team_Indianapolis Colts,Home Team_Jacksonville Jaguars,Home Team_Kansas City Chiefs,Home Team_Los Angeles Chargers,Home Team_Los Angeles Rams,Home Team_Miami Dolphins,Home Team_Minnesota Vikings,Home Team_New England Patriots,Home Team_New Orleans Saints,Home Team_New York Giants,Home Team_New York Jets,Home Team_Oakland Raiders,Home Team_Philadelphia Eagles,Home Team_Pittsburgh Steelers,Home Team_San Francisco 49ers,Home Team_Seattle Seahawks,Home Team_Tampa Bay Buccaneers,Home Team_Tennessee Titans,Home Team_Washington Football Team,Away Team_Arizona Cardinals,Away Team_Atlanta Falcons,Away Team_Baltimore Ravens,Away Team_Buffalo Bills,Away Team_Carolina Panthers,Away Team_Chicago Bears,Away Team_Cincinnati Bengals,Away Team_Cleveland Browns,Away Team_Dallas Cowboys,Away Team_Denver Broncos,Away Team_Detroit Lions,Away Team_Green Bay Packers,Away Team_Houston Texans,Away Team_Indianapolis Colts,Away Team_Jacksonville Jaguars,Away Team_Kansas City Chiefs,Away Team_Los Angeles Chargers,Away Team_Los Angeles Rams,Away Team_Miami Dolphins,Away Team_Minnesota Vikings,Away Team_New England Patriots,Away Team_New Orleans Saints,Away Team_New York Giants,Away Team_New York Jets,Away Team_Oakland Raiders,Away Team_Philadelphia Eagles,Away Team_Pittsburgh Steelers,Away Team_San Francisco 49ers,Away Team_Seattle Seahawks,Away Team_Tampa Bay Buccaneers,Away Team_Tennessee Titans,Away Team_Washington Football Team,team_score,opp_score
3228,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,20.0,31.0
3229,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,12.0,20.0
3235,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,24.0,17.0
3238,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,14.0,16.0
3243,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,20.0,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3433,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,18.0,26.0
3437,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,31.0,38.0
3440,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,27.0,28.0
3447,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,24.0,10.0


In [27]:
xt=trainData.drop(columns=['team_score','opp_score']).to_numpy()
xt=torch.from_numpy(xt).float()

yt=trainData[['team_score','opp_score']].to_numpy()
yt=torch.from_numpy(yt).float()

print(xt[0])
yt

tensor([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])


tensor([[34., 23.],
        [34.,  0.],
        [31., 24.],
        [21., 18.],
        [30., 27.],
        [20., 17.],
        [ 9., 21.],
        [37., 42.],
        [29., 29.],
        [31., 20.],
        [20., 13.],
        [20., 19.],
        [27., 21.],
        [21., 17.],
        [ 6., 27.],
        [31., 21.],
        [24., 13.],
        [27., 14.],
        [31., 17.],
        [35., 23.],
        [37., 43.],
        [22., 27.],
        [28., 20.],
        [38., 27.],
        [ 6.,  9.],
        [27., 30.],
        [38., 31.],
        [14., 26.],
        [22.,  0.],
        [36., 37.],
        [34., 37.],
        [18., 33.],
        [26., 23.],
        [17., 20.],
        [23., 27.],
        [38., 24.],
        [41., 17.],
        [12.,  9.],
        [19., 16.],
        [34., 16.],
        [31., 23.],
        [31., 33.],
        [21., 23.],
        [26., 10.],
        [13., 12.],
        [13., 34.],
        [27., 17.],
        [ 0., 21.],
        [23., 17.],
        [31., 28.],


In [28]:
xv=valData.drop(columns=['team_score','opp_score']).to_numpy()
xv=torch.from_numpy(xv).float()

yv=valData[['team_score','opp_score']].to_numpy()
yv=torch.from_numpy(yv).float()

print(xv[0])
yv

tensor([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])


tensor([[20., 31.],
        [12., 20.],
        [24., 17.],
        [14., 16.],
        [20., 16.],
        [26., 10.],
        [45., 42.],
        [26., 24.],
        [38.,  7.],
        [31., 12.],
        [29., 27.],
        [48., 10.],
        [18., 28.],
        [30., 14.],
        [27., 17.],
        [33., 31.],
        [43., 19.],
        [20., 13.],
        [26., 23.],
        [20., 19.],
        [23., 20.],
        [36., 21.],
        [14., 28.],
        [21., 37.],
        [17., 25.],
        [45., 35.],
        [16., 23.],
        [28., 16.],
        [34., 22.],
        [34., 10.],
        [14., 51.],
        [21., 23.],
        [10., 24.],
        [33., 40.],
        [30., 33.],
        [41., 17.],
        [30., 16.],
        [26., 23.],
        [20., 12.],
        [13., 16.],
        [26., 18.],
        [38., 31.],
        [28., 27.],
        [10., 24.],
        [ 0., 31.],
        [31., 20.],
        [20., 12.],
        [17., 24.],
        [16., 14.],
        [16., 20.],


### principal

In [ ]:
#defino modelo a lo rocho
qFactors=10
tamaniodered=10

cero=torch.zeros([32,qFactors])
def parametros():
  #factors
  factors=torch.randn([32,qFactors],requires_grad=True)
  #forward
  w1=torch.randn([2*qFactors,tamaniodered],requires_grad=True)
  b1=torch.randn(tamaniodered,requires_grad=True)
  w2=torch.randn([tamaniodered,tamaniodered],requires_grad=True)
  b2=torch.randn(tamaniodered,requires_grad=True)
  w3=torch.randn([tamaniodered,2],requires_grad=True)
  b3=torch.randn(2,requires_grad=True)

  return [factors,w1,w2,w3,b1,b2,b3]

def model(x,parameters):
  #defino parametros
  factors=parameters[0]
  w1=parameters[1]
  w2=parameters[2]
  w3=parameters[3]
  b1=parameters[4]
  b2=parameters[5]
  b3=parameters[6]

  F1 = x @ torch.cat((factors,cero))
  F2 = x @ torch.cat((cero,factors))
  inputs= torch.cat((F1,F2),axis=1)

  #hago la mini red
  layer1=torch.nn.functional.relu(inputs @ w1 + b1)
  layer2=torch.nn.functional.relu(layer1 @ w2 + b2)
  return torch.nn.functional.relu(layer2 @ w3 + b3)

fito=stochFit(fitDF,stochNum=100,epochNum=1000,lr=0.1,graph=True,wd=2)


print(mse(model(xt,fito['bestParams']),yt))
mse(model(xv,fito['bestParams']),yv)

tensor(69.8979)


tensor(63.9534)

### chequeo el accuracy

In [ ]:
df2=pd.get_dummies(df1.loc[df1['season']==2019],columns=['Home Team','Away Team'])
df2

,season,week,elo1_pre,elo2_pre,elo_prob1,Home Odds Open,y,team_score,opp_score,time,scoreDiff,Home Team_Arizona Cardinals,Home Team_Atlanta Falcons,Home Team_Baltimore Ravens,Home Team_Buffalo Bills,Home Team_Carolina Panthers,Home Team_Chicago Bears,Home Team_Cincinnati Bengals,Home Team_Cleveland Browns,Home Team_Dallas Cowboys,Home Team_Denver Broncos,Home Team_Detroit Lions,Home Team_Green Bay Packers,Home Team_Houston Texans,Home Team_Indianapolis Colts,Home Team_Jacksonville Jaguars,Home Team_Kansas City Chiefs,Home Team_Los Angeles Chargers,Home Team_Los Angeles Rams,Home Team_Miami Dolphins,Home Team_Minnesota Vikings,Home Team_New England Patriots,Home Team_New Orleans Saints,Home Team_New York Giants,Home Team_New York Jets,Home Team_Oakland Raiders,Home Team_Philadelphia Eagles,Home Team_Pittsburgh Steelers,Home Team_San Francisco 49ers,Home Team_Seattle Seahawks,Home Team_Tampa Bay Buccaneers,Home Team_Tennessee Titans,Home Team_Washington Football Team,Away Team_Arizona Cardinals,Away Team_Atlanta Falcons,Away Team_Baltimore Ravens,Away Team_Buffalo Bills,Away Team_Carolina Panthers,Away Team_Chicago Bears,Away Team_Cincinnati Bengals,Away Team_Cleveland Browns,Away Team_Dallas Cowboys,Away Team_Denver Broncos,Away Team_Detroit Lions,Away Team_Green Bay Packers,Away Team_Houston Texans,Away Team_Indianapolis Colts,Away Team_Jacksonville Jaguars,Away Team_Kansas City Chiefs,Away Team_Los Angeles Chargers,Away Team_Los Angeles Rams,Away Team_Miami Dolphins,Away Team_Minnesota Vikings,Away Team_New England Patriots,Away Team_New Orleans Saints,Away Team_New York Giants,Away Team_New York Jets,Away Team_Oakland Raiders,Away Team_Philadelphia Eagles,Away Team_Pittsburgh Steelers,Away Team_San Francisco 49ers,Away Team_Seattle Seahawks,Away Team_Tampa Bay Buccaneers,Away Team_Tennessee Titans,Away Team_Washington Football Team
3471,2019.0,1.0,1588.897931,1455.131370,0.758449,0.632911,0,3.0,10.0,217.0,-7.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3472,2019.0,1.0,1538.423954,1520.315825,0.617372,0.666667,1,28.0,12.0,217.0,16.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3473,2019.0,1.0,1415.178762,1570.539249,0.372816,0.333333,0,10.0,59.0,217.0,-49.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3474,2019.0,1.0,1384.966442,1448.423682,0.502220,0.625000,0,16.0,17.0,217.0,-1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3475,2019.0,1.0,1565.411649,1428.533515,0.761715,0.813008,1,21.0,20.0,217.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3722,2019.0,17.0,1351.254830,1551.327497,0.314850,0.350877,0,17.0,34.0,233.0,-17.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3723,2019.0,17.0,1744.319584,1523.483714,0.838269,0.546448,1,28.0,10.0,233.0,18.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3724,2019.0,17.0,1570.662276,1609.709252,0.537280,0.434783,0,21.0,26.0,233.0,-5.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3725,2019.0,17.0,1584.215388,1542.863113,0.648445,0.357143,0,14.0,35.0,233.0,-21.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [ ]:
#tengo que hacer un predict en el elo2

test=df2.loc[df2['time']==sem]
XX=test[xteam2].to_numpy()
XX=torch.from_numpy(XX).float()
test['score1Hat']=model(XX,fito['bestParams'])[:,0].detach().numpy()
test['score2Hat']=model(XX,fito['bestParams'])[:,1].detach().numpy()

#rate en primer semana
test['yHat_1factor']=0
test.loc[test['score1Hat']>test['score2Hat'],'yHat_1factor']=1
rateOfSucc(test,'yHat_1factor')


TypeError: ignored

In [ ]:
test['yHat_elo']=0
test.loc[test['elo1_pre']>test['elo2_pre'],'yHat_elo']=1
rateOfSucc(test,'yHat_elo')

### fit and bag

In [137]:
#defino modelo a lo rocho
qFactors=10
tamaniodered=10

cero=torch.zeros([32,qFactors])
def parametros():
  #factors
  factors=torch.randn([32,qFactors],requires_grad=True)
  #forward
  w1=torch.randn([2*qFactors,tamaniodered],requires_grad=True)
  b1=torch.randn(tamaniodered,requires_grad=True)
  w2=torch.randn([tamaniodered,tamaniodered],requires_grad=True)
  b2=torch.randn(tamaniodered,requires_grad=True)
  w3=torch.randn([tamaniodered,2],requires_grad=True)
  b3=torch.randn(2,requires_grad=True)

  return [factors,w1,w2,w3,b1,b2,b3]

def model(x,parameters):
  #defino parametros
  factors=parameters[0]
  w1=parameters[1]
  w2=parameters[2]
  w3=parameters[3]
  b1=parameters[4]
  b2=parameters[5]
  b3=parameters[6]

  F1 = x @ torch.cat((factors,cero))
  F2 = x @ torch.cat((cero,factors))
  inputs= torch.cat((F1,F2),axis=1)

  #hago la mini red
  layer1=torch.nn.functional.relu(inputs @ w1 + b1)
  layer2=torch.nn.functional.relu(layer1 @ w2 + b2)
  return torch.nn.functional.relu(layer2 @ w3 + b3)

fito=fitindebag(fitDF,stochNum=10,epochNum=10,lr=0.1,graph=True,wd=0)


px.histogram(x=[dictio['bestError'] for dictio in fito]).show()
px.scatter(x=[t for t in range(0,len(fito))],y=[dictio['bestError'] for dictio in fito]).show()


In [103]:

def modelFYB(x,fito):
  result=model(x,fito[0]['bestParams'])
  for dicti in fito:
    result=result+model(x,dicti['bestParams'])
  return result/(len(fito)+1)
def modelFYBPercent(x,fito,valError=100):
  errores=np.array([i['bestError'] for i in fito])

  #encuentro los valores menor a percent
  indices=np.where(errores<valError)[0]
  
  
  return modelFYB(XX,[fito[iid] for iid in indices])
  
def modelFYBcontrol(x,fito,valError=0.2,bins=0.1):
  errores=np.array([i['bestError'] for i in fito])
  minimo=np.amin(errores)
  maximo=np.amax(errores)

  upperBound=(valError+bins)*(maximo-minimo)+minimo
  lowerBound=(valError-bins)*(maximo-minimo)+minimo

  #encuentro los valores menor a percent
  indices=np.where((errores<upperBound) & (errores>lowerBound))[0]
  
  
  return modelFYB(XX,[fito[iid] for iid in indices])
  


### chequeo el accuracy

In [113]:
df2=pd.get_dummies(df1.loc[df1['season']==2019],columns=['Home Team','Away Team'])
df2

,season,week,elo1_pre,elo2_pre,elo_prob1,Home Odds Open,y,team_score,opp_score,time,scoreDiff,Home Team_Arizona Cardinals,Home Team_Atlanta Falcons,Home Team_Baltimore Ravens,Home Team_Buffalo Bills,Home Team_Carolina Panthers,Home Team_Chicago Bears,Home Team_Cincinnati Bengals,Home Team_Cleveland Browns,Home Team_Dallas Cowboys,Home Team_Denver Broncos,Home Team_Detroit Lions,Home Team_Green Bay Packers,Home Team_Houston Texans,Home Team_Indianapolis Colts,Home Team_Jacksonville Jaguars,Home Team_Kansas City Chiefs,Home Team_Los Angeles Chargers,Home Team_Los Angeles Rams,Home Team_Miami Dolphins,Home Team_Minnesota Vikings,Home Team_New England Patriots,Home Team_New Orleans Saints,Home Team_New York Giants,Home Team_New York Jets,Home Team_Oakland Raiders,Home Team_Philadelphia Eagles,Home Team_Pittsburgh Steelers,Home Team_San Francisco 49ers,Home Team_Seattle Seahawks,Home Team_Tampa Bay Buccaneers,Home Team_Tennessee Titans,Home Team_Washington Football Team,Away Team_Arizona Cardinals,Away Team_Atlanta Falcons,Away Team_Baltimore Ravens,Away Team_Buffalo Bills,Away Team_Carolina Panthers,Away Team_Chicago Bears,Away Team_Cincinnati Bengals,Away Team_Cleveland Browns,Away Team_Dallas Cowboys,Away Team_Denver Broncos,Away Team_Detroit Lions,Away Team_Green Bay Packers,Away Team_Houston Texans,Away Team_Indianapolis Colts,Away Team_Jacksonville Jaguars,Away Team_Kansas City Chiefs,Away Team_Los Angeles Chargers,Away Team_Los Angeles Rams,Away Team_Miami Dolphins,Away Team_Minnesota Vikings,Away Team_New England Patriots,Away Team_New Orleans Saints,Away Team_New York Giants,Away Team_New York Jets,Away Team_Oakland Raiders,Away Team_Philadelphia Eagles,Away Team_Pittsburgh Steelers,Away Team_San Francisco 49ers,Away Team_Seattle Seahawks,Away Team_Tampa Bay Buccaneers,Away Team_Tennessee Titans,Away Team_Washington Football Team
3471,2019.0,1.0,1588.897931,1455.131370,0.758449,0.632911,0,3.0,10.0,217.0,-7.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3472,2019.0,1.0,1538.423954,1520.315825,0.617372,0.666667,1,28.0,12.0,217.0,16.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3473,2019.0,1.0,1415.178762,1570.539249,0.372816,0.333333,0,10.0,59.0,217.0,-49.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3474,2019.0,1.0,1384.966442,1448.423682,0.502220,0.625000,0,16.0,17.0,217.0,-1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3475,2019.0,1.0,1565.411649,1428.533515,0.761715,0.813008,1,21.0,20.0,217.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3722,2019.0,17.0,1351.254830,1551.327497,0.314850,0.350877,0,17.0,34.0,233.0,-17.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3723,2019.0,17.0,1744.319584,1523.483714,0.838269,0.546448,1,28.0,10.0,233.0,18.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3724,2019.0,17.0,1570.662276,1609.709252,0.537280,0.434783,0,21.0,26.0,233.0,-5.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3725,2019.0,17.0,1584.215388,1542.863113,0.648445,0.357143,0,14.0,35.0,233.0,-21.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [114]:
#tengo que hacer un predict en el elo2

test=df2.loc[df2['time']==sem]
XX=test[xteam2].to_numpy()
XX=torch.from_numpy(XX).float()
test['score1Hat']=modelFYB(XX,fito)[:,0].detach().numpy()
test['score2Hat']=modelFYB(XX,fito)[:,1].detach().numpy()

#rate en primer semana
test['yHat_1factor']=0
test.loc[test['score1Hat']>test['score2Hat'],'yHat_1factor']=1
rateOfSucc(test,'yHat_1factor')


0.625

In [115]:
test['yHat_elo']=0
test.loc[test['elo1_pre']>test['elo2_pre'],'yHat_elo']=1
rateOfSucc(test,'yHat_elo')

0.625

### val error vs test error

In [118]:
erroresdeval=[]
erroresdetest=[]
for dictionario in fito:
  erroresdeval.append(dictionario['bestError'])
  test=df2.loc[df2['time']==sem]
  XX=test[xteam2].to_numpy()
  XX=torch.from_numpy(XX).float()
  test['score1Hat']=model(XX,dictionario['bestParams'])[:,0].detach().numpy()
  test['score2Hat']=model(XX,dictionario['bestParams'])[:,1].detach().numpy()

  #rate en primer semana
  test['yHat_1factor']=0
  test.loc[test['score1Hat']>test['score2Hat'],'yHat_1factor']=1
  erroresdetest.append(rateOfSucc(test,'yHat_1factor'))

px.scatter(x=erroresdeval,y=erroresdetest).show()

In [121]:

#hago el test para baggin
espacio=np.linspace(88,120)
baggeosTest=[]
for vali in espacio:
  
  test=df2.loc[df2['time']==sem]
  XX=test[xteam2].to_numpy()
  XX=torch.from_numpy(XX).float()
  test['score1Hat']=modelFYBPercent(XX,fito,valError=vali)[:,0].detach().numpy()
  test['score2Hat']=modelFYBPercent(XX,fito,valError=vali)[:,1].detach().numpy()

  #rate en primer semana
  test['yHat_1factor']=0
  test.loc[test['score1Hat']>test['score2Hat'],'yHat_1factor']=1
  baggeosTest.append(rateOfSucc(test,'yHat_1factor'))
px.line(x=espacio,y=baggeosTest).show()

### all year 

In [134]:
def getTensors(df1,sem,valPercent=0.8,bac=17):
  data=df1.loc[(df1['time']>sem-bac) & (df1['time']<sem)]

  data['split']=np.random.binomial(1, valPercent, len(data))
  data1=data.copy()
  data1['Home Team']=data['Away Team']
  data1['Away Team']=data['Home Team']
  data1['team_score']=data['opp_score']
  data1['opp_score']=data['team_score']
  data=pd.concat([data,data1])
  tecData=pd.get_dummies(data,columns=['Home Team','Away Team'])
  trainData=tecData.loc[tecData['split']==1]
  valData=tecData.loc[tecData['split']==0]
  trainData=trainData[xteam2+['team_score','opp_score']]
  valData=valData[xteam2+['team_score','opp_score']]

  xt=trainData.drop(columns=['team_score','opp_score']).to_numpy()
  xt=torch.from_numpy(xt).float()

  yt=trainData[['team_score','opp_score']].to_numpy()
  yt=torch.from_numpy(yt).float()
  return {'xt':xt,'xv':xv,'yt':yt,'yv':yv}

def modelRateWeek(modelo,params):
  test=df2.loc[df2['time']==sem]
  XX=test[xteam2].to_numpy()
  XX=torch.from_numpy(XX).float()
  test['score1Hat']=modelo(XX,params)[:,0].detach().numpy()
  test['score2Hat']=modelo(XX,params)[:,1].detach().numpy()

  #rate en primer semana
  test['yHat_1factor']=0
  test.loc[test['score1Hat']>test['score2Hat'],'yHat_1factor']=1

  test['yHat_elo']=0
  test.loc[test['elo1_pre']>test['elo2_pre'],'yHat_elo']=1
  
  return {'mio':rateOfSucc(test,'yHat_1factor'),'elo':rateOfSucc(test,'yHat_elo'),'test':test}

In [125]:
#primero con volatilidad
testArray=[]
for sem in range(217,234):
  dat=getTensors(df1,sem)
  xt=dat['xt']
  xv=dat['xv']
  yt=dat['yt']
  yv=dat['yv']

  qFactors=10
  tamaniodered=10

  #el modelo esta definido arriba
  fito=stochFit(fitDF,stochNum=10,epochNum=2000,lr=0.1,graph=True,wd=2)
  print('mio ',modelRateWeek(model,fito['bestParams'])['mio'])
  print('elo ',modelRateWeek(model,fito['bestParams'])['elo'])
  testArray.appande(modelRateWeek(model,fito['bestParams'])['test'])


  
finaTest=pd.concat(testArray)
display(finaTest)
print(rateOfSucc(finaTest,'yHat_1factor'))
print(rateOfSucc(finaTest,'yHat_elo'))


mio  0.6875
elo  0.8125


mio  0.75
elo  0.625


mio  0.4375
elo  0.6875


mio  0.6666666666666666
elo  0.7333333333333333


mio  0.3333333333333333
elo  0.5333333333333333


mio  0.5714285714285714
elo  0.5


mio  0.7857142857142857
elo  0.6428571428571429


mio  0.8
elo  0.8666666666666667


KeyboardInterrupt: ignored

In [ ]:
#primero con volatilidad
testArray=[]
for sem in range(217,234):
  dat=getTensors(df1,sem)
  xt=dat['xt']
  xv=dat['xv']
  yt=dat['yt']
  yv=dat['yv']

  qFactors=10
  tamaniodered=10

  #el modelo esta definido arriba
  fito=fitindebag(fitDF,stochNum=100,epochNum=2000,lr=0.1,graph=True,wd=2)
  main=modelRateWeek(modelFYB,fito)
  print(sem)
  print('mio ',main['mio'])
  print('elo ',main['elo'])
  testArray.append(main['test'])
  print('------------------')


  
finaTest=pd.concat(testArray)
display(finaTest)
print(rateOfSucc(finaTest,'yHat_1factor'))
print(rateOfSucc(finaTest,'yHat_elo'))



217
mio  0.8125
elo  0.8125
------------------
218
mio  0.625
elo  0.625
------------------


# --------------------only time---------------------------

In [ ]:
#variables de las ultimas n seasons, (maximo 2 seasons de viejo)
#info upto the point y training up to the point
#las series temporales son super diferentes
#uso solo datos de t-1
#hay una version leave one out mas cheta del modelo pero es demasiado
#idea que tengo es usar entrenamiento de las ultimas n temporadas y hacer un predict de toda una temporada
#back en 6 season es un mini estandart

# ultra simple

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifier

## 2019

### logistic

In [ ]:
back=30

eloTec=pd.get_dummies(elo1, columns=['team1',	'team2'])
eloTecTrain=eloTec.loc[(eloTec['season']<2019) & (eloTec['season']>=2019-back)]
eloTecTest=eloTec.loc[eloTec['season']==2019]
eloTecTrain

,season,elo1_pre,elo2_pre,elo_prob1,y,split,team1_AKR,team1_ARI,team1_ATL,team1_BAL,team1_BBA,team1_BCL,team1_BDA,team1_BFF,team1_BKN,team1_BRL,team1_BUF,team1_BYK,team1_CAR,team1_CBD,team1_CHB,team1_CHI,team1_CHL,team1_CHT,team1_CIB,team1_CIN,team1_CLE,team1_CLI,team1_COL,team1_CRA,team1_CRP,team1_CST,team1_CTI,team1_DAL,team1_DAY,team1_DEN,team1_DET,team1_DHR,team1_DPN,team1_DTI,...,team2_MUT,team2_NAA,team2_NE,team2_NG1,team2_NO,team2_NYA,team2_NYG,team2_NYJ,team2_NYY,team2_OAK,team2_OOR,team2_PHI,team2_PIT,team2_PRV,team2_PTB,team2_PTQ,team2_RAC,team2_RCH,team2_RED,team2_RII,team2_ROS,team2_SEA,team2_SF,team2_SIS,team2_SLA,team2_STG,team2_STP,team2_SYR,team2_TB,team2_TEN,team2_THO,team2_TLM,team2_TOL,team2_TON,team2_TOR,team2_UTI,team2_WBU,team2_WHE,team2_WJA,team2_WSH
8546,1989,1568.685000,1555.235000,0.611018,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
8547,1989,1463.497000,1454.383000,0.605070,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8548,1989,1523.626000,1528.083000,0.586257,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8549,1989,1549.138000,1528.584000,0.620692,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8550,1989,1460.109000,1556.001000,0.455660,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16269,2018,1640.171960,1647.624483,0.582068,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
16270,2018,1669.105861,1633.114673,0.641378,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
16271,2018,1682.450194,1648.424105,0.638772,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
16272,2018,1675.286412,1661.668566,0.611248,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
y=eloTecTrain.loc[:,'y'].to_numpy()
X=eloTecTrain.loc[:,xelo+xteam].to_numpy()
clf = LogisticRegression().fit(X, y)

predict1['logistic_time']=clf.predict(eloTec.loc[:,xelo+xteam].to_numpy())
predict1
print(rateOfSucc(predict1.loc[predict1['season']==2019],'logistic_time'))

NameError: ignored

In [ ]:
#chequeo el mejor back
backy=[n for n in range(1,99)]
accuracys=[]
for back in backy:


  eloTec=pd.get_dummies(elo1, columns=['team1',	'team2'])
  eloTecTrain=eloTec.loc[(eloTec['season']<2019) & (eloTec['season']>=2019-back)]
  eloTecTest=eloTec.loc[eloTec['season']==2019]

  y=eloTecTrain.loc[:,'y'].to_numpy()
  X=eloTecTrain.loc[:,xelo+xteam].to_numpy()
  clf = LogisticRegression().fit(X, y)

  predict1['logistic_time']=clf.predict(eloTec.loc[:,xelo+xteam].to_numpy())
  accuracys.append(rateOfSucc(predict1.loc[predict1['season']==2019],'logistic_time'))
  #print(rateOfSucc(predict1.loc[predict1['season']==2019],'logistic_time'))
px.line(x=backy,y=accuracys).show()

### ridge

In [ ]:
#hago el ridge model, malla de backs=[3,15,55]
back=3

training=((eloTec['season']<2019) & (eloTec['season']>=2019-back)) &  (eloTec['split']==1)
validating=((eloTec['season']<2019) & (eloTec['season']>=2019-back)) &  (eloTec['split']==0)
trainAccArray=[]
testAccArray=[]
net=np.logspace(-4, 7)

eloTec=pd.get_dummies(elo1, columns=['team1',	'team2'])
eloTecTrain=eloTec.loc[training]
eloTecTest=eloTec.loc[eloTec['season']==2019]
y=eloTecTrain.loc[:,'y'].to_numpy()
X=eloTecTrain.loc[:,xelo+xteam].to_numpy()


for regu in net:
  clf = RidgeClassifier(alpha=regu).fit(X, y)
  predict1['ridge']=clf.predict(eloTec.loc[:,xelo+xteam].to_numpy())
  
  trainAccArray.append(rateOfSucc(predict1.loc[training],'ridge'))
  testAccArray.append(rateOfSucc(predict1.loc[validating],'ridge'))

  
px.line(x=list(net)+list(net),
        y=testAccArray+trainAccArray,
        log_x=True).show()


#chequeo el parametro piola
clf = RidgeClassifier(alpha=24).fit(X, y)
predict1['ridge']=clf.predict(eloTec.loc[:,xelo+xteam].to_numpy())
print(rateOfSucc(predict1.loc[predict1['season']==2019],'ridge'))

0.6591760299625468


### forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
net

array([   10.,    11.,    13.,    15.,    17.,    20.,    23.,    26.,
          30.,    35.,    40.,    47.,    54.,    62.,    71.,    82.,
          95.,   109.,   126.,   145.,   167.,   193.,   222.,   255.,
         294.,   339.,   390.,   449.,   517.,   596.,   686.,   790.,
         910.,  1048.,  1206.,  1389.,  1599.,  1842.,  2120.,  2442.,
        2811.,  3237.,  3727.,  4291.,  4941.,  5689.,  6551.,  7543.,
        8685., 10000.])

In [ ]:
#hago el ridge model, malla de backs=[3,15,55]
back=3

training=((eloTec['season']<2019) & (eloTec['season']>=2019-back)) &  (eloTec['split']==1)
validating=((eloTec['season']<2019) & (eloTec['season']>=2019-back)) &  (eloTec['split']==0)
trainAccArray=[]
testAccArray=[]
#net=[n for n in range(2,30)]
net=np.logspace(-4,4)#.astype(int)

eloTec=pd.get_dummies(elo1, columns=['team1',	'team2'])
eloTecTrain=eloTec.loc[training]
eloTecTest=eloTec.loc[eloTec['season']==2019]
y=eloTecTrain.loc[:,'y'].to_numpy()
X=eloTecTrain.loc[:,xelo+xteam].to_numpy()


for regu in net:
  clf = RandomForestClassifier(min_impurity_decrease=regu,n_jobs=-1).fit(X, y)
  predict1['ridge']=clf.predict(eloTec.loc[:,xelo+xteam].to_numpy())
  
  trainAccArray.append(rateOfSucc(predict1.loc[training],'ridge'))
  testAccArray.append(rateOfSucc(predict1.loc[validating],'ridge'))

  
px.line(x=list(net)+list(net),
        y=testAccArray+trainAccArray,
        log_x=False).show()


#chequeo el parametro piola
clf = RandomForestClassifier(min_samples_split=222).fit(X, y)
predict1['ridge']=clf.predict(eloTec.loc[:,xelo+xteam].to_numpy())
print(rateOfSucc(predict1.loc[predict1['season']==2019],'ridge'))

0.6329588014981273


In [ ]:
# el numero este me permite crear una mala, del tipo n/2,n,n*2
# max depth, 4, 0.63
# min_samples_split 222 , 0.65
#min_samples_leaf, 10 o 11, 0.64
#max_leaf_nodes 9,4,18, 0.65
# max_leaf_nodes 4 0.66


In [ ]:
#hago el ridge model, malla de backs=[3,15,55]
back=3

training=((eloTec['season']<2019) & (eloTec['season']>=2019-back)) &  (eloTec['split']==1)
validating=((eloTec['season']<2019) & (eloTec['season']>=2019-back)) &  (eloTec['split']==0)
trainAccArray=[]
testAccArray=[]
#net=[n for n in range(2,30)]
msp=[111,222,444]
msl=[5,10,20]
mln=[2,4,8]

eloTec=pd.get_dummies(elo1, columns=['team1',	'team2'])
eloTecTrain=eloTec.loc[training]
eloTecTest=eloTec.loc[eloTec['season']==2019]
y=eloTecTrain.loc[:,'y'].to_numpy()
X=eloTecTrain.loc[:,xelo+xteam].to_numpy()


for one in msp:
  for two in msl:
    for three in mln:
      clf = RandomForestClassifier(min_samples_split=one,min_samples_leaf=two,max_leaf_nodes=three,n_jobs=-1).fit(X, y)
      predict1['ridge']=clf.predict(eloTec.loc[:,xelo+xteam].to_numpy())
      
      trainAccArray.append(rateOfSucc(predict1.loc[training],'ridge'))
      testAccArray.append(rateOfSucc(predict1.loc[validating],'ridge'))

  
px.line(x=[n for n in range(0,27)],
        y=testAccArray,
        log_x=False).show()


#chequeo el parametro piola
clf = RandomForestClassifier(min_samples_split=222).fit(X, y)
predict1['ridge']=clf.predict(eloTec.loc[:,xelo+xteam].to_numpy())
print(rateOfSucc(predict1.loc[predict1['season']==2019],'ridge'))

0.6367041198501873


### svc

In [ ]:
from sklearn.svm import SVC

In [ ]:
#hago el ridge model, malla de backs=[3,15,55]
back=3

training=((eloTec['season']<2019) & (eloTec['season']>=2019-back)) &  (eloTec['split']==1)
validating=((eloTec['season']<2019) & (eloTec['season']>=2019-back)) &  (eloTec['split']==0)
trainAccArray=[]
testAccArray=[]
net=np.logspace(-4, 6)

eloTec=pd.get_dummies(elo1, columns=['team1',	'team2'])
eloTecTrain=eloTec.loc[training]
eloTecTest=eloTec.loc[eloTec['season']==2019]
y=eloTecTrain.loc[:,'y'].to_numpy()
X=eloTecTrain.loc[:,xelo+xteam].to_numpy()

counter=0
for regu in net:
  counter+=1
  print(counter, end='')
  clf = SVC(C=regu,kernel='sigmoid').fit(X, y)
  predict1['ridge']=clf.predict(eloTec.loc[:,xelo+xteam].to_numpy())
  
  trainAccArray.append(rateOfSucc(predict1.loc[training],'ridge'))
  testAccArray.append(rateOfSucc(predict1.loc[validating],'ridge'))

  
px.line(x=list(net)+list(net),
        y=testAccArray+trainAccArray,
        log_x=True).show()


#chequeo el parametro piola
clf = SVC(C=1000,kernel='poly').fit(X, y)
predict1['ridge']=clf.predict(eloTec.loc[:,xelo+xteam].to_numpy())
print(rateOfSucc(predict1.loc[predict1['season']==2019],'ridge'))

1234567891011121314151617181920212223242526272829303132333435363738394041424344454647484950

0.651685393258427


In [ ]:
#C 
#kernel
# rbf pega al palo a 0.66
#poly 0.68

### adaboost MAL

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
#hago el ridge model, malla de backs=[3,15,55]
back=3

training=((eloTec['season']<2019) & (eloTec['season']>=2019-back)) &  (eloTec['split']==1)
validating=((eloTec['season']<2019) & (eloTec['season']>=2019-back)) &  (eloTec['split']==0)
trainAccArray=[]
testAccArray=[]
net=np.logspace(-4, 0)
#net=[n for n in range(1,100)]

eloTec=pd.get_dummies(elo1, columns=['team1',	'team2'])
eloTecTrain=eloTec.loc[training]
eloTecTest=eloTec.loc[eloTec['season']==2019]
y=eloTecTrain.loc[:,'y'].to_numpy()
X=eloTecTrain.loc[:,xelo+xteam].to_numpy()

counter=0
for regu in net:
  counter+=1
  print(counter, end='')
  clf = AdaBoostClassifier(n_estimators=10,learning_rate=regu).fit(X, y)
  predict1['ridge']=clf.predict(eloTec.loc[:,xelo+xteam].to_numpy())
  
  trainAccArray.append(rateOfSucc(predict1.loc[training],'ridge'))
  testAccArray.append(rateOfSucc(predict1.loc[validating],'ridge'))

  
px.line(x=list(net)+list(net),
        y=testAccArray+trainAccArray,
        log_x=True).show()


#chequeo el parametro piola
clf = SVC(C=1000,kernel='poly').fit(X, y)
predict1['ridge']=clf.predict(eloTec.loc[:,xelo+xteam].to_numpy())
print(rateOfSucc(predict1.loc[predict1['season']==2019],'ridge'))

1234567891011121314151617181920212223242526272829303132333435363738394041424344454647484950

0.651685393258427


In [ ]:
#n_estimators

### net

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
#hago el ridge model, malla de backs=[3,15,55]
back=3

training=((eloTec['season']<2019) & (eloTec['season']>=2019-back)) &  (eloTec['split']==1)
validating=((eloTec['season']<2019) & (eloTec['season']>=2019-back)) &  (eloTec['split']==0)
trainAccArray=[]
testAccArray=[]
#net=[1e-5,1,10]
#net=[n for n in range(1,100)]
net=np.logspace(-4, 1)

eloTec=pd.get_dummies(elo1, columns=['team1',	'team2'])
eloTecTrain=eloTec.loc[training]
eloTecTest=eloTec.loc[eloTec['season']==2019]
y=eloTecTrain.loc[:,'y'].to_numpy()
X=eloTecTrain.loc[:,xelo+xteam].to_numpy()

counter=0
for regu in net:
  counter+=1
  print(counter, end='')
  clf = MLPClassifier(solver='adam', alpha=regu,hidden_layer_sizes=(8, 1),activation='tanh').fit(X, y)
  predict1['ridge']=clf.predict(eloTec.loc[:,xelo+xteam].to_numpy())
  
  trainAccArray.append(rateOfSucc(predict1.loc[training],'ridge'))
  testAccArray.append(rateOfSucc(predict1.loc[validating],'ridge'))

  
px.line(x=list(net)+list(net),
        y=testAccArray+trainAccArray,
        log_x=True).show()


#chequeo el parametro piola
clf = SVC(C=1000,kernel='poly').fit(X, y)
predict1['ridge']=clf.predict(eloTec.loc[:,xelo+xteam].to_numpy())
print(rateOfSucc(predict1.loc[predict1['season']==2019],'ridge'))

1234567

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.



8910111213141516

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.



17181920

...

49

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.



50

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.



0.651685393258427


In [ ]:
#la optimizacion es muy random

In [ ]:
#n_estimators